### Student name: Nguyen Xuan Tung
### Student ID: M22.ICT.006
### Version1 : Apply Histogram equalization on grayscale image
### It is recommended to restart the notebook after running each cell to get consistent results and avoid out-of-memory errors.

### HE

In [2]:
from __future__ import annotations
import numpy as np
from numba import cuda, core
from PIL import Image
import requests
from io import BytesIO

class HistogramEqualization(object):
    _cache = {}
    _cache_2 = {}
    _cache_3 = {}
    _cache_4 = {}
    _cache_5 = {}

    _WARP_SIZE = 32
    _NUM_WARPS = 8

    def __init__(self, functor):
        self._functor = functor

    @staticmethod
    def _gpu_kernel_factory():
        def reshape_3d_to_2d(d_input_orig, d_input):
            x, y = cuda.grid(2)
            height = d_input_orig.shape[0]
            width = d_input_orig.shape[1]
            if x < width and y < height:
                idx = y * width + x
                d_input[idx][0] = d_input_orig[y, x][0]
                d_input[idx][1] = d_input_orig[y, x][1]
                d_input[idx][2] = d_input_orig[y, x][2]

        return cuda.jit(reshape_3d_to_2d)

    def _compile(self, dtype):
        key = self._functor, dtype
        if key not in self._cache:
            self._cache[key] = HistogramEqualization._gpu_kernel_factory()
        return self._cache[key]

    @staticmethod
    def _gpu_kernel_factory_2():
        max_block_size = (
            HistogramEqualization._NUM_WARPS * HistogramEqualization._WARP_SIZE
        )

        @cuda.jit(device=True)
        def load_shared_memory_input(
            shared_memory_r, shared_memory_g, shared_memory_b, d_input
        ):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)

            shared_memory_r[local_tid] = d_input[global_tid][0]
            shared_memory_g[local_tid] = d_input[global_tid][1]
            shared_memory_b[local_tid] = d_input[global_tid][2]

            cuda.syncthreads()

        @cuda.jit(device=True)
        def load_shared_memory_gray(shared_memory_gray, d_output):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)
            shared_memory_gray[local_tid] = d_output[global_tid]
            cuda.syncthreads()

        @cuda.jit(device=True)
        def load_shared_memory_cdf(histogram, cdf):
            local_tid = cuda.threadIdx.x
            cdf[local_tid] = histogram[local_tid]
            cuda.syncthreads()

        @cuda.jit(device=True)
        def grayscale_image(
            shared_memory_r, shared_memory_g, shared_memory_b, d_output
        ):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)
            grayscale_value = round(
                shared_memory_r[local_tid] * 0.299
                + shared_memory_g[local_tid] * 0.587
                + shared_memory_b[local_tid] * 0.114
            )
            d_output[global_tid] = grayscale_value

        @cuda.jit(device=True)
        def save_shared_memory_histogram(shared_memory_histogram, d_hist):
            tid = cuda.threadIdx.x
            cuda.syncthreads()
            while tid < max_block_size:
                cuda.atomic.add(d_hist, tid, shared_memory_histogram[tid])
                tid += cuda.blockDim.x

        @cuda.jit(device=True)
        def init_shared_memory_histogram(shared_memory_histogram):
            local_tid = cuda.threadIdx.x
            shared_memory_histogram[local_tid] = 0
            cuda.syncthreads()

        def histogram(d_input, d_output, d_hist, d_cdf):
            shared_memory_red = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )
            shared_memory_green = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )
            shared_memory_blue = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )
            shared_memory_gray = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )

            load_shared_memory_input(
                shared_memory_red, shared_memory_green, shared_memory_blue, d_input
            )

            grayscale_image(
                shared_memory_red, shared_memory_green, shared_memory_blue, d_output
            )

            shared_memory_histogram = cuda.shared.array(shape=256, dtype=np.float32)

            load_shared_memory_gray(shared_memory_gray, d_output)
            init_shared_memory_histogram(shared_memory_histogram)
            tid = cuda.grid(1)
            gridDim = cuda.gridDim.x * cuda.blockDim.x
            while tid < d_output.size:
                pixel_values = d_output[tid]
                cuda.atomic.add(shared_memory_histogram, pixel_values, 1)
                tid += gridDim
            save_shared_memory_histogram(shared_memory_histogram, d_hist)

            load_shared_memory_cdf(d_hist, d_cdf)

        return cuda.jit(histogram)

    def _compile_2(self, dtype):
        key = dtype
        if key not in self._cache_2:
            self._cache_2[key] = HistogramEqualization._gpu_kernel_factory_2()
        return self._cache_2[key]

    @staticmethod
    def _gpu_kernel_factory_3(fn):
        scan_op = cuda.jit(device=True)(fn)

        max_block_size = (
            HistogramEqualization._NUM_WARPS * HistogramEqualization._WARP_SIZE
        )

        @cuda.jit(device=True)
        def pointer_jumping(cdf, jump):
            tid = cuda.threadIdx.x

            right = tid + jump
            temp = cdf[tid]
            cuda.syncthreads()
            if right < max_block_size:
                cdf[right] = scan_op(temp, cdf[right])
            cuda.syncthreads()

        @cuda.jit(device=True)
        def calculate_cdf(cdf) -> None:
            local_tid = cuda.threadIdx.x
            jump = 1
            while jump < cuda.blockDim.x:
                pointer_jumping(cdf, jump)
                jump *= 2
            cuda.syncthreads()
            max_value = cdf[-1]
            if max_value > 0 and local_tid < max_block_size:
                temp = cdf[local_tid]
                cdf[local_tid] = (temp * 255) // max_value
            cuda.syncthreads()

        def cdf(d_cdf):
            calculate_cdf(d_cdf)

        return cuda.jit(cdf)

    def _compile_3(self, dtype):
        key = self._functor, dtype
        if key not in self._cache_3:
            self._cache_3[key] = HistogramEqualization._gpu_kernel_factory_3(
                self._functor
            )
        return self._cache_3[key]

    @staticmethod
    def _gpu_kernel_factory_4():
        @cuda.jit(device=True)
        def calculate_histogram_equalization(cdf, d_output):
            global_tid = cuda.grid(1)
            temp = d_output[global_tid]
            d_output[global_tid] = cdf[temp]
            cuda.syncthreads()

        def histogram_equalization(cdf, d_output):
            calculate_histogram_equalization(cdf, d_output)

        return cuda.jit(histogram_equalization)

    def _compile_4(self, dtype):
        key = self._functor, dtype
        if key not in self._cache_4:
            self._cache_4[key] = HistogramEqualization._gpu_kernel_factory_4()
        return self._cache_4[key]

    @staticmethod
    def _gpu_kernel_factory_5():
        def reshape_1d_to_2d(d_input, d_output):
            global_tid = cuda.grid(1)
            width = d_output.shape[1]
            if global_tid < d_input.size:
                row = global_tid // width
                column = global_tid % width

                d_output[row, column] = d_input[global_tid]

        return cuda.jit(reshape_1d_to_2d)

    def _compile_5(self, dtype):
        key = self._functor, dtype
        if key not in self._cache_5:
            self._cache_5[key] = HistogramEqualization._gpu_kernel_factory_5()
        return self._cache_5[key]

    def __call__(self, d_input_orig):
        height, width = d_input_orig.shape[0], d_input_orig.shape[1]
        max_block_size = (
            HistogramEqualization._WARP_SIZE * HistogramEqualization._NUM_WARPS
        )
        stream = cuda.default_stream()

        _sav, core.config.CUDA_LOW_OCCUPANCY_WARNINGS = (
            core.config.CUDA_LOW_OCCUPANCY_WARNINGS,
            False,
        )
        kernel = self._compile(d_input_orig.dtype)
        kernel2 = self._compile_2(d_input_orig.dtype)
        kernel3 = self._compile_3(d_input_orig.dtype)
        kernel4 = self._compile_4(d_input_orig.dtype)
        kernel5 = self._compile_5(d_input_orig.dtype)

        d_input = cuda.device_array(
            shape=(height * width, 3), dtype=np.float32, stream=stream
        )
        d_final_output_1d = cuda.device_array(
            shape=(height * width), dtype=np.uint8, stream=stream
        )
        d_final_output_2d = cuda.device_array(
            shape=(height, width), dtype=np.uint8, stream=stream
        )
        d_hist = cuda.device_array(shape=(256,), dtype=np.float32, stream=stream)
        d_cdf = cuda.device_array(shape=(256,), dtype=np.float32, stream=stream)

        start_event = cuda.event(True)
        start_event.record(stream=stream)
        nb_threads = max_block_size

        nb_threads = (16, 16)
        nb_blocks = (
            (width + nb_threads[1] - 1) // nb_threads[1],
            (height + nb_threads[0] - 1) // nb_threads[0],
        )
        kernel[nb_blocks, nb_threads, stream](d_input_orig, d_input)

        nb_threads = min(max_block_size, d_input.size)
        nb_blocks = (d_input.shape[0] + nb_threads - 1) // nb_threads

        kernel2[nb_blocks, nb_threads, stream](
            d_input, d_final_output_1d, d_hist, d_cdf
        )
        kernel3[1, nb_threads, stream](d_cdf)
        kernel4[nb_blocks, nb_threads, stream](d_cdf, d_final_output_1d)
        kernel5[nb_blocks, nb_threads, stream](d_final_output_1d, d_final_output_2d)
        stop_event = cuda.event(True)
        stop_event.record(stream=stream)
        stop_event.synchronize()
        core.config.CUDA_LOW_OCCUPANCY_WARNINGS = _sav

        return (
            cuda.event_elapsed_time(start_event, stop_event),
            d_final_output_2d.copy_to_host(),
        )


if __name__ == "__main__":

    def histogram_equalization():
        he = HistogramEqualization(lambda a, b: a + b)
        image_url = "https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/000cf025-48d6-45b2-987f-c19db5250699/width=1024/image%20(18).jpeg"
        response = requests.get(image_url)
        image_bytes = BytesIO(response.content)
        image = Image.open(image_bytes)
        image = np.array(image, dtype=np.float32)
        d_input_image = cuda.to_device(image)
        ct = []
        for i in range(6):
            if i == 0:
                time, result_image = he(d_input_image)
                result_image = Image.fromarray(result_image)
                result_image.save("he.png")
            else:
                time, _ = he(d_input_image)
            ct.append(time)
        print(f"average kernel computation time is {np.average(ct[1:])} ms")
        print(ct)

    histogram_equalization()


average kernel computation time is 0.31411840319633483 ms
[570.285400390625, 0.44521600008010864, 0.2789120078086853, 0.2861120104789734, 0.28195199370384216, 0.2784000039100647]


### AHE

In [1]:
from __future__ import annotations

import numpy as np
from numba import cuda, core
from PIL import Image
import requests
from io import BytesIO

class AdjustedHistogramEqualization(object):
    _cache = {}
    _cache_2 = {}
    _cache_3 = {}
    _cache_4 = {}
    _cache_5 = {}
    _cache_6 = {}
    _cache_7 = {}

    _WARP_SIZE = 32
    _NUM_WARPS = 8

    def __init__(self, functor):
        self._functor = functor

    @staticmethod
    def _gpu_kernel_factory():
        def reshape_3d_to_2d(d_input_orig, d_input):
            x, y = cuda.grid(2)
            height = d_input_orig.shape[0]
            width = d_input_orig.shape[1]
            if x < width and y < height:
                idx = y * width + x
                d_input[idx][0] = d_input_orig[y, x][0]
                d_input[idx][1] = d_input_orig[y, x][1]
                d_input[idx][2] = d_input_orig[y, x][2]

        return cuda.jit(reshape_3d_to_2d)

    def _compile(self, dtype):
        key = self._functor, dtype
        if key not in self._cache:
            self._cache[key] = AdjustedHistogramEqualization._gpu_kernel_factory()
        return self._cache[key]

    @staticmethod
    def _gpu_kernel_factory_2():
        max_block_size = (
            AdjustedHistogramEqualization._NUM_WARPS
            * AdjustedHistogramEqualization._WARP_SIZE
        )

        @cuda.jit(device=True)
        def load_shared_memory_input(
            shared_memory_r, shared_memory_g, shared_memory_b, d_input
        ):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)

            shared_memory_r[local_tid] = d_input[global_tid][0]
            shared_memory_g[local_tid] = d_input[global_tid][1]
            shared_memory_b[local_tid] = d_input[global_tid][2]

            cuda.syncthreads()

        @cuda.jit(device=True)
        def load_shared_memory_gray(shared_memory_gray, d_output):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)
            shared_memory_gray[local_tid] = d_output[global_tid]
            cuda.syncthreads()

        @cuda.jit(device=True)
        def grayscale_image(
            shared_memory_r, shared_memory_g, shared_memory_b, d_output
        ):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)
            grayscale_value = round(
                shared_memory_r[local_tid] * 0.299
                + shared_memory_g[local_tid] * 0.587
                + shared_memory_b[local_tid] * 0.114
            )
            d_output[global_tid] = grayscale_value

        @cuda.jit(device=True)
        def save_shared_memory_histogram(shared_memory_histogram, d_hist):
            tid = cuda.threadIdx.x
            cuda.syncthreads()
            while tid < max_block_size:
                cuda.atomic.add(d_hist, tid, shared_memory_histogram[tid])
                tid += cuda.blockDim.x

        @cuda.jit(device=True)
        def init_shared_memory_histogram(shared_memory_histogram):
            local_tid = cuda.threadIdx.x
            shared_memory_histogram[local_tid] = 0
            cuda.syncthreads()
            

        def histogram(d_input, d_output, d_hist):
            shared_memory_red = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )
            shared_memory_green = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )
            shared_memory_blue = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )
            shared_memory_gray = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )

            load_shared_memory_input(
                shared_memory_red, shared_memory_green, shared_memory_blue, d_input
            )

            grayscale_image(
                shared_memory_red, shared_memory_green, shared_memory_blue, d_output
            )

            shared_memory_histogram = cuda.shared.array(shape=256, dtype=np.float32)

            load_shared_memory_gray(shared_memory_gray, d_output)
            init_shared_memory_histogram(shared_memory_histogram)
            tid = cuda.grid(1)
            gridDim = cuda.gridDim.x * cuda.blockDim.x
            while tid < d_output.size:
                pixel_values = d_output[tid]
                cuda.atomic.add(shared_memory_histogram, pixel_values, 1)
                tid += gridDim
            save_shared_memory_histogram(shared_memory_histogram, d_hist)

        return cuda.jit(histogram)

    def _compile_2(self, dtype):
        key = dtype
        if key not in self._cache_2:
            self._cache_2[key] = AdjustedHistogramEqualization._gpu_kernel_factory_2()
        return self._cache_2[key]

    @staticmethod
    def _gpu_kernel_factory_3():
        @cuda.jit(device=True)
        def create_uniform_histogram(d_uniform_hist, number_of_pixels):
            local_tid = cuda.threadIdx.x

            pixels_per_bin = number_of_pixels // 256

            remainder = number_of_pixels - pixels_per_bin * 256

            d_uniform_hist[local_tid] = pixels_per_bin

            if local_tid < remainder:
                d_uniform_hist[local_tid] += 1

        def uniform_histogram(d_uniform_hist, number_of_pixels):
            create_uniform_histogram(d_uniform_hist, number_of_pixels)

        return cuda.jit(uniform_histogram)

    def _compile_3(self, dtype):
        key = dtype
        if key not in self._cache_3:
            self._cache_3[key] = AdjustedHistogramEqualization._gpu_kernel_factory_3()
        return self._cache_3[key]

    @staticmethod
    def _gpu_kernel_factory_4():
        @cuda.jit(device=True)
        def calculate_adjusted_histogram(
            d_uniform_hist, d_hist, d_adjusted_hist, lambda_value
        ):
            local_tid = cuda.threadIdx.x
            d_adjusted_hist[local_tid] = (1 / (1 + lambda_value)) * d_hist[
                local_tid
            ] + (lambda_value / (1 + lambda_value)) * d_uniform_hist[local_tid]

        @cuda.jit(device=True)
        def load_shared_memory_cdf(histogram, cdf):
            local_tid = cuda.threadIdx.x
            cdf[local_tid] = histogram[local_tid]
            cuda.syncthreads()

        def adjusted_histogram(
            d_uniform_hist, d_hist, d_adjusted_hist, lambda_value, d_cdf
        ):
            calculate_adjusted_histogram(
                d_uniform_hist, d_hist, d_adjusted_hist, lambda_value
            )
            load_shared_memory_cdf(d_adjusted_hist, d_cdf)

        return cuda.jit(adjusted_histogram)

    def _compile_4(self, dtype):
        key = dtype
        if key not in self._cache_4:
            self._cache_4[key] = AdjustedHistogramEqualization._gpu_kernel_factory_4()
        return self._cache_4[key]

    @staticmethod
    def _gpu_kernel_factory_5(fn):
        scan_op = cuda.jit(device=True)(fn)

        max_block_size = (
            AdjustedHistogramEqualization._NUM_WARPS
            * AdjustedHistogramEqualization._WARP_SIZE
        )

        @cuda.jit(device=True)
        def pointer_jumping(cdf, jump):
            tid = cuda.threadIdx.x

            right = tid + jump
            temp = cdf[tid]
            cuda.syncthreads()
            if right < max_block_size:
                cdf[right] = scan_op(temp, cdf[right])
            cuda.syncthreads()

        @cuda.jit(device=True)
        def calculate_cdf(cdf) -> None:
            local_tid = cuda.threadIdx.x
            jump = 1
            while jump < cuda.blockDim.x:
                pointer_jumping(cdf, jump)
                jump *= 2
            cuda.syncthreads()
            max_value = cdf[-1]
            if max_value > 0 and local_tid < max_block_size:
                temp = cdf[local_tid]
                cdf[local_tid] = (temp * 255) // max_value
            cuda.syncthreads()

        def cdf(d_cdf):
            calculate_cdf(d_cdf)

        return cuda.jit(cdf)

    def _compile_5(self, dtype):
        key = self._functor, dtype
        if key not in self._cache_5:
            self._cache_5[key] = AdjustedHistogramEqualization._gpu_kernel_factory_5(
                self._functor
            )
        return self._cache_5[key]

    @staticmethod
    def _gpu_kernel_factory_6():
        @cuda.jit(device=True)
        def calculate_histogram_equalization(cdf, d_output):
            global_tid = cuda.grid(1)
            temp = d_output[global_tid]
            d_output[global_tid] = cdf[temp]
            cuda.syncthreads()

        def histogram_equalization(cdf, d_output):
            calculate_histogram_equalization(cdf, d_output)

        return cuda.jit(histogram_equalization)

    def _compile_6(self, dtype):
        key = self._functor, dtype
        if key not in self._cache_6:
            self._cache_6[key] = AdjustedHistogramEqualization._gpu_kernel_factory_6()
        return self._cache_6[key]

    @staticmethod
    def _gpu_kernel_factory_7():
        def reshape_1d_to_2d(d_input, d_output):
            global_tid = cuda.grid(1)
            width = d_output.shape[1]
            if global_tid < d_input.size:
                row = global_tid // width
                column = global_tid % width

                d_output[row, column] = d_input[global_tid]

        return cuda.jit(reshape_1d_to_2d)

    def _compile_7(self, dtype):
        key = self._functor, dtype
        if key not in self._cache_7:
            self._cache_7[key] = AdjustedHistogramEqualization._gpu_kernel_factory_7()
        return self._cache_7[key]

    def __call__(self, d_input_orig, lambda_value):
        height, width = d_input_orig.shape[0], d_input_orig.shape[1]
        max_block_size = (
            AdjustedHistogramEqualization._WARP_SIZE
            * AdjustedHistogramEqualization._NUM_WARPS
        )
        stream = cuda.default_stream()

        nb_threads = min(max_block_size, d_input_orig.size)

        _sav, core.config.CUDA_LOW_OCCUPANCY_WARNINGS = (
            core.config.CUDA_LOW_OCCUPANCY_WARNINGS,
            False,
        )
        number_of_pixels = height * width
        d_input = cuda.device_array(
            shape=(height * width, 3), dtype=np.float32, stream=stream
        )
        d_hist = cuda.device_array(shape=(256,), dtype=np.float32, stream=stream)
        d_cdf = cuda.device_array(shape=(256,), dtype=np.float32, stream=stream)
        d_uniform_hist = cuda.device_array(
            shape=(256,), dtype=np.float32, stream=stream
        )
        d_adjusted_hist = cuda.device_array(
            shape=(256,), dtype=np.float32, stream=stream
        )
        d_final_output_1d = cuda.device_array(
            shape=(height * width), dtype=np.uint8, stream=stream
        )
        d_final_output_2d = cuda.device_array(
            shape=(height, width), dtype=np.uint8, stream=stream
        )

        kernel = self._compile(d_input_orig.dtype)
        kernel2 = self._compile_2(d_input_orig.dtype)
        kernel3 = self._compile_3(
            d_input_orig.dtype,
        )
        kernel4 = self._compile_4(d_input_orig.dtype)
        kernel5 = self._compile_5(d_input_orig.dtype)
        kernel6 = self._compile_6(d_input_orig.dtype)
        kernel7 = self._compile_7(d_input_orig.dtype)

        start_event = cuda.event(True)
        start_event.record(stream=stream)

        nb_threads = (16, 16)
        nb_blocks = (
            (width + nb_threads[1] - 1) // nb_threads[1],
            (height + nb_threads[0] - 1) // nb_threads[0],
        )
        kernel[nb_blocks, nb_threads, stream](d_input_orig, d_input)

        nb_threads = min(max_block_size, d_input_orig.size)
        nb_blocks = (d_input.shape[0] + nb_threads - 1) // nb_threads

        kernel2[nb_blocks, nb_threads, stream](d_input, d_final_output_1d, d_hist)
        kernel3[1, nb_threads, stream](d_uniform_hist, number_of_pixels)
        kernel4[1, nb_threads, stream](
            d_uniform_hist, d_hist, d_adjusted_hist, lambda_value, d_cdf
        )
        kernel5[1, nb_threads, stream](d_cdf)
        kernel6[nb_blocks, nb_threads, stream](d_cdf, d_final_output_1d)
        kernel7[nb_blocks, nb_threads, stream](d_final_output_1d, d_final_output_2d)

        stop_event = cuda.event(True)
        stop_event.record(stream=stream)
        stop_event.synchronize()

        core.config.CUDA_LOW_OCCUPANCY_WARNINGS = _sav

        return (
            cuda.event_elapsed_time(start_event, stop_event),
            d_final_output_2d.copy_to_host(),
        )


if __name__ == "__main__":

    def adjusted_histogram_equalization():
        ahe = AdjustedHistogramEqualization(lambda a, b: a + b)
        image_url = "https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/000cf025-48d6-45b2-987f-c19db5250699/width=1024/image%20(18).jpeg"
        response = requests.get(image_url)
        image_bytes = BytesIO(response.content)
        image = Image.open(image_bytes)
        image = np.array(image, dtype=np.float32)

        d_input_image = cuda.to_device(image)
        lambda_value = 3
        ct = []
        for i in range(6):
            if i == 0:
                time, result_image = ahe(d_input_image, lambda_value)
                result_image = Image.fromarray(result_image)
                result_image.save("ahe.png")
            else:
                time, _ = ahe(d_input_image, lambda_value)
            ct.append(time)
        print(f"average kernel computation time is {np.average(ct[1:])} ms")
        print(ct)

    adjusted_histogram_equalization()


average kernel computation time is 0.3777919948101044 ms
[568.291748046875, 0.5232959985733032, 0.34463998675346375, 0.3468799889087677, 0.33478400111198425, 0.33935999870300293]


### WHE

In [1]:
from __future__ import annotations
import numpy as np
from numba import cuda, core
from PIL import Image
import requests
from io import BytesIO

class WeightedHistogramEqualization(object):
    _cache = {}
    _cache_2 = {}
    _cache_3 = {}
    _cache_4 = {}
    _cache_5 = {}
    _cache_6 = {}
    _cache_7 = {}
    _cache_8 = {}
    _cache_9 = {}
    _cache_10 = {}

    _WARP_SIZE = 32
    _NUM_WARPS = 8

    def __init__(self, functor):
        self._functor = functor

    @staticmethod
    def _gpu_kernel_factory():
        def reshape_3d_to_2d(d_input_orig, d_input):
            x, y = cuda.grid(2)
            height = d_input_orig.shape[0]
            width = d_input_orig.shape[1]
            if x < width and y < height:
                idx = y * width + x
                d_input[idx][0] = d_input_orig[y, x][0]
                d_input[idx][1] = d_input_orig[y, x][1]
                d_input[idx][2] = d_input_orig[y, x][2]

        return cuda.jit(reshape_3d_to_2d)

        return cuda.jit(reshape)

    def _compile(self, dtype):
        key = self._functor, dtype
        if key not in self._cache:
            self._cache[key] = WeightedHistogramEqualization._gpu_kernel_factory()
        return self._cache[key]

    @staticmethod
    def _gpu_kernel_factory_2():
        max_block_size = (
            WeightedHistogramEqualization._NUM_WARPS
            * WeightedHistogramEqualization._WARP_SIZE
        )

        @cuda.jit(device=True)
        def load_shared_memory_input(
            shared_memory_r, shared_memory_g, shared_memory_b, d_input
        ):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)

            shared_memory_r[local_tid] = d_input[global_tid][0]
            shared_memory_g[local_tid] = d_input[global_tid][1]
            shared_memory_b[local_tid] = d_input[global_tid][2]

            cuda.syncthreads()

        @cuda.jit(device=True)
        def load_shared_memory_gray(shared_memory_gray, d_output):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)
            shared_memory_gray[local_tid] = d_output[global_tid]
            cuda.syncthreads()

        @cuda.jit(device=True)
        def grayscale_image(
            shared_memory_r, shared_memory_g, shared_memory_b, d_output
        ):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)
            grayscale_value = round(
                shared_memory_r[local_tid] * 0.299
                + shared_memory_g[local_tid] * 0.587
                + shared_memory_b[local_tid] * 0.114
            )
            d_output[global_tid] = grayscale_value

        @cuda.jit(device=True)
        def save_shared_memory_histogram(shared_memory_histogram, d_hist):
            tid = cuda.threadIdx.x
            cuda.syncthreads()
            while tid < max_block_size:
                cuda.atomic.add(d_hist, tid, shared_memory_histogram[tid])
                tid += cuda.blockDim.x

        @cuda.jit(device=True)
        def init_shared_memory_histogram(shared_memory_histogram):
            local_tid = cuda.threadIdx.x
            shared_memory_histogram[local_tid] = 0
            cuda.syncthreads()

        def histogram(d_input, d_output, d_hist):
            shared_memory_red = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )
            shared_memory_green = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )
            shared_memory_blue = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )
            shared_memory_gray = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )

            load_shared_memory_input(
                shared_memory_red, shared_memory_green, shared_memory_blue, d_input
            )

            grayscale_image(
                shared_memory_red, shared_memory_green, shared_memory_blue, d_output
            )

            shared_memory_histogram = cuda.shared.array(shape=256, dtype=np.float32)

            load_shared_memory_gray(shared_memory_gray, d_output)
            init_shared_memory_histogram(shared_memory_histogram)
            tid = cuda.grid(1)
            gridDim = cuda.gridDim.x * cuda.blockDim.x
            while tid < d_output.size:
                pixel_values = d_output[tid]
                cuda.atomic.add(shared_memory_histogram, pixel_values, 1)
                tid += gridDim
            save_shared_memory_histogram(shared_memory_histogram, d_hist)

        return cuda.jit(histogram)

    def _compile_2(self, dtype):
        key = dtype
        if key not in self._cache_2:
            self._cache_2[key] = WeightedHistogramEqualization._gpu_kernel_factory_2()
        return self._cache_2[key]

    @staticmethod
    def _gpu_kernel_factory_3():
        @cuda.jit
        def create_uniform_histogram(d_uniform_hist, number_of_pixels):
            local_tid = cuda.threadIdx.x

            pixels_per_bin = number_of_pixels // 256

            remainder = number_of_pixels - pixels_per_bin * 256

            d_uniform_hist[local_tid] = pixels_per_bin

            if local_tid < remainder:
                d_uniform_hist[local_tid] += 1

        def uniform_histogram(d_uniform_hist, number_of_pixels):
            create_uniform_histogram(d_uniform_hist, number_of_pixels)

        return cuda.jit(uniform_histogram)

    def _compile_3(self, dtype):
        key = dtype
        if key not in self._cache_3:
            self._cache_3[key] = WeightedHistogramEqualization._gpu_kernel_factory_3()

        return self._cache_3[key]

    @staticmethod
    def _gpu_kernel_factory_4():
        def calculate_local_variance(d_output, d_local_var, height, width, window_size):
            x, y = cuda.grid(2)
            radius = window_size // 2

            sum_pixels = 0.0
            sum_pixels_squared = 0.0
            num_pixels = 0
            for dy in range(-radius, radius + 1):
                for dx in range(-radius, radius + 1):
                    new_x = x + dx
                    new_y = y + dy
                    if new_x >= 0 and new_x < width and new_y >= 0 and new_y < height:
                        pixel_value = d_output[new_y * width + new_x]
                        sum_pixels += pixel_value
                        sum_pixels_squared += pixel_value**2
                        num_pixels += 1
                    else:
                        continue
            mean = sum_pixels / num_pixels
            variance = (sum_pixels_squared / num_pixels) - (mean**2)
            d_local_var[y * width + x] = variance
            cuda.syncthreads()

        return cuda.jit(calculate_local_variance)

    def _compile_4(self, dtype):
        key = dtype
        if key not in self._cache_4:
            self._cache_4[key] = WeightedHistogramEqualization._gpu_kernel_factory_4()

        return self._cache_4[key]

    @staticmethod
    def _gpu_kernel_factory_5():
        def track_local_variance(d_output, d_local_var, d_total_var):
            tid = cuda.grid(1)
            if tid < d_output.size:
                var = d_local_var[tid]
                pixel_intensity = d_output[tid]
                cuda.atomic.add(d_total_var, pixel_intensity, var)
            cuda.syncthreads()

        return cuda.jit(track_local_variance)

    def _compile_5(self, dtype):
        key = dtype
        if key not in self._cache_5:
            self._cache_5[key] = WeightedHistogramEqualization._gpu_kernel_factory_5()

        return self._cache_5[key]

    @staticmethod
    def _gpu_kernel_factory_6():
        def average_variance(d_average_local_var, d_total_var, d_hist):
            tid = cuda.grid(1)
            if d_total_var[tid] == 0:
                d_average_local_var[tid] = 0
            else:
                d_average_local_var[tid] = d_total_var[tid] / d_hist[tid]

        return cuda.jit(average_variance)

    def _compile_6(self, dtype):
        key = dtype
        if key not in self._cache_6:
            self._cache_6[key] = WeightedHistogramEqualization._gpu_kernel_factory_6()

        return self._cache_6[key]

    @staticmethod
    def _gpu_kernel_factory_7():
        @cuda.jit(device=True)
        def calculate_modified_hist(
            d_average_local_var, d_hist, d_uniform_hist, d_modified_hist, lambda_value
        ):
            tid = cuda.grid(1)
            d_modified_hist[tid] = (
                d_average_local_var[tid] * d_hist[tid]
                + lambda_value * d_uniform_hist[tid]
            ) / (d_average_local_var[tid] + lambda_value)
            cuda.syncthreads()

        def modified_hist(
            d_average_local_var, d_hist, d_uniform_hist, d_modified_hist, lambda_value
        ):
            calculate_modified_hist(
                d_average_local_var,
                d_hist,
                d_uniform_hist,
                d_modified_hist,
                lambda_value,
            )

        return cuda.jit(modified_hist)

    def _compile_7(self, dtype):
        key = dtype
        if key not in self._cache_7:
            self._cache_7[key] = WeightedHistogramEqualization._gpu_kernel_factory_7()

        return self._cache_7[key]

    @staticmethod
    def _gpu_kernel_factory_8(fn):
        scan_op = cuda.jit(device=True)(fn)

        max_block_size = (
            WeightedHistogramEqualization._NUM_WARPS
            * WeightedHistogramEqualization._WARP_SIZE
        )

        @cuda.jit(device=True)
        def pointer_jumping(cdf, jump):
            tid = cuda.threadIdx.x

            right = tid + jump
            temp = cdf[tid]
            cuda.syncthreads()
            if right < max_block_size:
                cdf[right] = scan_op(temp, cdf[right])
            cuda.syncthreads()

        @cuda.jit(device=True)
        def calculate_cdf(cdf) -> None:
            local_tid = cuda.threadIdx.x
            jump = 1
            while jump < cuda.blockDim.x:
                pointer_jumping(cdf, jump)
                jump *= 2
            cuda.syncthreads()
            max_value = cdf[-1]
            if max_value > 0 and local_tid < max_block_size:
                temp = cdf[local_tid]
                cdf[local_tid] = (temp * 255) // max_value
            cuda.syncthreads()

        def cdf(d_cdf):
            calculate_cdf(d_cdf)

        return cuda.jit(cdf)

    def _compile_8(self, dtype):
        key = self._functor, dtype
        if key not in self._cache_8:
            self._cache_8[key] = WeightedHistogramEqualization._gpu_kernel_factory_8(
                self._functor
            )
        return self._cache_8[key]

    @staticmethod
    def _gpu_kernel_factory_9():
        @cuda.jit(device=True)
        def calculate_weighted_histogram_equalization(cdf, d_output):
            global_tid = cuda.grid(1)
            temp = d_output[global_tid]
            d_output[global_tid] = cdf[temp]
            cuda.syncthreads()

        def weighted_histogram_equalization(cdf, d_output):
            calculate_weighted_histogram_equalization(cdf, d_output)

        return cuda.jit(weighted_histogram_equalization)

    @staticmethod
    def _gpu_kernel_factory_10():
        def reshape_1d_to_2d(d_input, d_output):
            global_tid = cuda.grid(1)
            width = d_output.shape[1]
            if global_tid < d_input.size:
                row = global_tid // width
                column = global_tid % width

                d_output[row, column] = d_input[global_tid]

        return cuda.jit(reshape_1d_to_2d)

    def _compile_10(self, dtype):
        key = self._functor, dtype
        if key not in self._cache_10:
            self._cache_10[key] = WeightedHistogramEqualization._gpu_kernel_factory_10()
        return self._cache_10[key]

    def _compile_9(self, dtype):
        key = self._functor, dtype
        if key not in self._cache_9:
            self._cache_9[key] = WeightedHistogramEqualization._gpu_kernel_factory_9()
        return self._cache_9[key]

    def __call__(self, d_input_orig, lambda_value, window_size):
        height, width = d_input_orig.shape[0], d_input_orig.shape[1]
        max_block_size = (
            WeightedHistogramEqualization._WARP_SIZE
            * WeightedHistogramEqualization._NUM_WARPS
        )
        stream = cuda.default_stream()

        _sav, core.config.CUDA_LOW_OCCUPANCY_WARNINGS = (
            core.config.CUDA_LOW_OCCUPANCY_WARNINGS,
            False,
        )
        d_input = cuda.device_array(
            shape=(height * width, 3), dtype=np.float32, stream=stream
        )
        d_final_output_1d = cuda.device_array(
            shape=(height * width), dtype=np.uint8, stream=stream
        )
        d_final_output_2d = cuda.device_array(
            shape=(height, width), dtype=np.uint8, stream=stream
        )
        number_of_pixels = d_input.size
        d_hist = cuda.device_array(shape=(256,), dtype=np.float32, stream=stream)
        d_uniform_hist = cuda.device_array(
            shape=(256,), dtype=np.float32, stream=stream
        )
        d_modified_hist = cuda.device_array(
            shape=(256,), dtype=np.float32, stream=stream
        )
        d_local_var = cuda.device_array(
            shape=(height * width), dtype=np.float32, stream=stream
        )
        d_total_var = cuda.device_array(shape=(256,), dtype=np.float32, stream=stream)
        d_average_local_var = cuda.device_array(
            shape=(256,), dtype=np.float32, stream=stream
        )

        kernel = self._compile(d_input_orig.dtype)
        kernel2 = self._compile_2(d_input_orig.dtype)
        kernel3 = self._compile_3(
            d_input_orig.dtype,
        )
        kernel4 = self._compile_4(d_input_orig.dtype)
        kernel5 = self._compile_5(d_input_orig.dtype)
        kernel6 = self._compile_6(d_input_orig.dtype)
        kernel7 = self._compile_7(d_input_orig.dtype)
        kernel8 = self._compile_8(d_input_orig.dtype)
        kernel9 = self._compile_9(d_input_orig.dtype)
        kernel10 = self._compile_10(d_input_orig.dtype)

        start_event = cuda.event(True)
        start_event.record(stream=stream)

        nb_threads = (16, 16)
        nb_blocks = (
            (width + nb_threads[1] - 1) // nb_threads[1],
            (height + nb_threads[0] - 1) // nb_threads[0],
        )
        kernel[nb_blocks, nb_threads, stream](d_input_orig, d_input)

        nb_threads = min(max_block_size, d_input_orig.size)
        nb_blocks = (d_input.shape[0] + nb_threads - 1) // nb_threads
        kernel2[nb_blocks, nb_threads, stream](d_input, d_final_output_1d, d_hist)
        kernel3[1, nb_threads, stream](d_uniform_hist, number_of_pixels)

        nb_threads = (16, 16)
        nb_blocks = (
            (width + nb_threads[1] - 1) // nb_threads[1],
            (height + nb_threads[0] - 1) // nb_threads[0],
        )
        kernel4[nb_blocks, nb_threads](
            d_final_output_1d, d_local_var, height, width, window_size
        )
        nb_threads = min(max_block_size, d_input_orig.size)
        nb_blocks = (d_input.shape[0] + nb_threads - 1) // nb_threads
        kernel5[nb_blocks, nb_threads, stream](
            d_final_output_1d, d_local_var, d_total_var
        )
        kernel6[1, nb_threads, stream](d_average_local_var, d_total_var, d_hist)
        kernel7[1, nb_threads, stream](
            d_average_local_var, d_hist, d_uniform_hist, d_modified_hist, lambda_value
        )
        kernel8[1, nb_threads, stream](d_modified_hist)
        kernel9[nb_blocks, nb_threads, stream](d_modified_hist, d_final_output_1d)
        kernel10[nb_blocks, nb_threads, stream](d_final_output_1d, d_final_output_2d)
        stop_event = cuda.event(True)
        stop_event.record(stream=stream)
        stop_event.synchronize()

        core.config.CUDA_LOW_OCCUPANCY_WARNINGS = _sav

        return (
            cuda.event_elapsed_time(start_event, stop_event),
            d_final_output_2d.copy_to_host(),
        )


if __name__ == "__main__":

    def weighted_histogram_equalization():
        whe = WeightedHistogramEqualization(lambda a, b: a + b)
        image_url = "https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/000cf025-48d6-45b2-987f-c19db5250699/width=1024/image%20(18).jpeg"
        response = requests.get(image_url)
        image_bytes = BytesIO(response.content)
        image = Image.open(image_bytes)
        image = np.array(image, dtype=np.float32)
        d_input_image = cuda.to_device(image)
        lambda_value = 3
        window_size = 3
        ct = []
        for i in range(6):
            if i == 0:
                time, result_image = whe(
                    d_input_image,
                    lambda_value,
                    window_size,
                )
                result_image = Image.fromarray(result_image)
                result_image.save("whe.png")
            else:
                time, _ = whe(
                    d_input_image,
                    lambda_value,
                    window_size,
                )
            ct.append(time)
        print(f"average kernel computation time is {np.average(ct[1:])} ms")
        print(ct)

    weighted_histogram_equalization()


average kernel computation time is 1.024889588356018 ms
[759.64306640625, 1.1565120220184326, 0.9907199740409851, 0.989471971988678, 0.9939519762992859, 0.9937919974327087]


### ESIHE

In [1]:
from __future__ import annotations

import numpy as np
from numba import cuda, core
from PIL import Image
import requests
from io import BytesIO

class ExposureHistogramEqualization(object):
    _cache = {}
    _cache_2 = {}
    _cache_3 = {}
    _cache_4 = {}
    _cache_5 = {}
    _cache_6 = {}
    _cache_7 = {}

    _WARP_SIZE = 32
    _NUM_WARPS = 8

    def __init__(self, functor):
        self._functor = functor

    @staticmethod
    def _gpu_kernel_factory():
        def reshape_3d_to_2d(d_input_orig, d_input):
            x, y = cuda.grid(2)
            height = d_input_orig.shape[0]
            width = d_input_orig.shape[1]
            if x < width and y < height:
                idx = y * width + x
                d_input[idx][0] = d_input_orig[y, x][0]
                d_input[idx][1] = d_input_orig[y, x][1]
                d_input[idx][2] = d_input_orig[y, x][2]

        return cuda.jit(reshape_3d_to_2d)

    def _compile(self, dtype):
        key = self._functor, dtype
        if key not in self._cache:
            self._cache[key] = ExposureHistogramEqualization._gpu_kernel_factory()
        return self._cache[key]

    @staticmethod
    def _gpu_kernel_factory_2():
        max_block_size = (
            ExposureHistogramEqualization._NUM_WARPS
            * ExposureHistogramEqualization._WARP_SIZE
        )

        @cuda.jit(device=True)
        def load_shared_memory_input(
            shared_memory_r, shared_memory_g, shared_memory_b, d_input
        ):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)

            shared_memory_r[local_tid] = d_input[global_tid][0]
            shared_memory_g[local_tid] = d_input[global_tid][1]
            shared_memory_b[local_tid] = d_input[global_tid][2]

            cuda.syncthreads()

        @cuda.jit(device=True)
        def load_shared_memory_gray(shared_memory_gray, d_output):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)
            shared_memory_gray[local_tid] = d_output[global_tid]
            cuda.syncthreads()

        @cuda.jit(device=True)
        def grayscale_image(
            shared_memory_r, shared_memory_g, shared_memory_b, d_output
        ):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)
            grayscale_value = round(
                shared_memory_r[local_tid] * 0.299
                + shared_memory_g[local_tid] * 0.587
                + shared_memory_b[local_tid] * 0.114
            )
            d_output[global_tid] = grayscale_value

        @cuda.jit(device=True)
        def save_shared_memory_histogram(shared_memory_histogram, d_hist):
            tid = cuda.threadIdx.x
            cuda.syncthreads()
            while tid < max_block_size:
                cuda.atomic.add(d_hist, tid, shared_memory_histogram[tid])
                tid += cuda.blockDim.x

        @cuda.jit(device=True)
        def init_shared_memory_histogram(shared_memory_histogram):
            local_tid = cuda.threadIdx.x
            shared_memory_histogram[local_tid] = 0
            cuda.syncthreads()

        def histogram(d_input, d_output, d_hist):
            shared_memory_red = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )
            shared_memory_green = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )
            shared_memory_blue = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )
            shared_memory_gray = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )

            load_shared_memory_input(
                shared_memory_red, shared_memory_green, shared_memory_blue, d_input
            )

            grayscale_image(
                shared_memory_red, shared_memory_green, shared_memory_blue, d_output
            )

            shared_memory_histogram = cuda.shared.array(shape=256, dtype=np.float32)

            load_shared_memory_gray(shared_memory_gray, d_output)
            init_shared_memory_histogram(shared_memory_histogram)
            tid = cuda.grid(1)
            gridDim = cuda.gridDim.x * cuda.blockDim.x
            while tid < d_output.size:
                pixel_values = d_output[tid]
                cuda.atomic.add(shared_memory_histogram, pixel_values, 1)
                tid += gridDim
            save_shared_memory_histogram(shared_memory_histogram, d_hist)

        return cuda.jit(histogram)

    def _compile_2(self, dtype):
        key = dtype
        if key not in self._cache_2:
            self._cache_2[key] = ExposureHistogramEqualization._gpu_kernel_factory_2()
        return self._cache_2[key]

    @staticmethod
    def _gpu_kernel_factory_3(fn):
        scan_op = cuda.jit(device=True)(fn)
        max_block_size = (
            ExposureHistogramEqualization._NUM_WARPS
            * ExposureHistogramEqualization._WARP_SIZE
        )

        @cuda.jit(device=True)
        def load_shared_memory(shared_memory, arr):
            local_tid = cuda.threadIdx.x
            if local_tid < arr.size:
                shared_memory[local_tid] = arr[local_tid]

        @cuda.jit(device=True)
        def pointer_jumping(cdf, jump):
            tid = cuda.threadIdx.x

            right = tid + jump
            temp = cdf[tid]
            cuda.syncthreads()
            if right < max_block_size:
                cdf[right] = scan_op(temp, cdf[right])
            cuda.syncthreads()

        @cuda.jit(device=True)
        def calculate_mean_histogram(d_hist, partials, mean_histogram, sum_histogram):
            load_shared_memory(partials, d_hist)

            jump = 1
            while jump < cuda.blockDim.x:
                pointer_jumping(partials, jump)
                jump = jump * 2

            if cuda.threadIdx.x == 0:
                sum_histogram[0] = partials[255]
                mean_histogram[0] = round(partials[255] / 256)

            cuda.syncthreads()

        @cuda.jit(device=True)
        def clip_histogram(d_hist, d_clip_hist, clip_value):
            local_tid = cuda.threadIdx.x
            if local_tid < max_block_size:
                temp = d_hist[local_tid]
                if temp > clip_value:
                    d_clip_hist[local_tid] = clip_value
                else:
                    d_clip_hist[local_tid] = temp

        def create_clip_histogram(d_hist, d_clip_hist, mean_histogram, sum_histogram):
            partials = cuda.shared.array(shape=256, dtype=np.float32)
            calculate_mean_histogram(d_hist, partials, mean_histogram, sum_histogram)
            clip_histogram(d_hist, d_clip_hist, mean_histogram[0])

        return cuda.jit(create_clip_histogram)

    def _compile_3(self, dtype):
        key = self._functor, dtype
        if key not in self._cache_3:
            self._cache_3[key] = ExposureHistogramEqualization._gpu_kernel_factory_3(
                self._functor
            )
        return self._cache_3[key]

    @staticmethod
    def _gpu_kernel_factory_4(fn):
        scan_op = cuda.jit(device=True)(fn)
        max_block_size = (
            ExposureHistogramEqualization._NUM_WARPS
            * ExposureHistogramEqualization._WARP_SIZE
        )

        @cuda.jit(device=True)
        def load_shared_memory(shared_memory, arr):
            local_tid = cuda.threadIdx.x
            if local_tid < arr.size:
                shared_memory[local_tid] = arr[local_tid]

        @cuda.jit(device=True)
        def pointer_jumping(cdf, jump):
            tid = cuda.threadIdx.x

            right = tid + jump
            temp = cdf[tid]
            cuda.syncthreads()
            if right < max_block_size:
                cdf[right] = scan_op(temp, cdf[right])
            cuda.syncthreads()

        @cuda.jit(device=True)
        def calculate_different_values(
            partials, sum_histogram, exposure, a_norm, x_mean
        ):
            jump = 1
            while jump < cuda.blockDim.x:
                pointer_jumping(partials, jump)
                jump = jump * 2

            if cuda.threadIdx.x == 0:
                exposure_value = (partials[255] / sum_histogram) / 256
                a_norm_value = 1 - exposure_value
                exposure[0] = exposure_value
                a_norm[0] = a_norm_value
                x_mean[0] = round(256 * a_norm_value)

            cuda.syncthreads()

        @cuda.jit(device=True)
        def multiply_index(
            partials,
        ):
            local_tid = cuda.threadIdx.x
            if local_tid < cuda.blockDim.x:
                temp = partials[local_tid]
                partials[local_tid] = temp * local_tid

            cuda.syncthreads()

        def exposure(d_hist, sum_histogram, exposure, a_norm, x_mean):
            partials = cuda.shared.array(shape=256, dtype=np.int32)
            load_shared_memory(partials, d_hist)

            multiply_index(partials)
            calculate_different_values(
                partials, sum_histogram, exposure, a_norm, x_mean
            )

        return cuda.jit(exposure)

    def _compile_4(self, dtype):
        key = self._functor, dtype
        if key not in self._cache_4:
            self._cache_4[key] = ExposureHistogramEqualization._gpu_kernel_factory_4(
                self._functor
            )
        return self._cache_4[key]

    @staticmethod
    def _gpu_kernel_factory_5(fn):
        scan_op = cuda.jit(device=True)(fn)
        max_block_size = (
            ExposureHistogramEqualization._NUM_WARPS
            * ExposureHistogramEqualization._WARP_SIZE
        )

        @cuda.jit(device=True)
        def load_shared_memory_left_hist(shared_memory, left_hist_size, arr):
            local_tid = cuda.threadIdx.x
            if local_tid < left_hist_size:
                shared_memory[local_tid] = arr[local_tid]

            cuda.syncthreads()

        @cuda.jit(device=True)
        def load_shared_memory_right_hist(shared_memory, right_hist_size, arr):
            local_tid = cuda.threadIdx.x
            if local_tid < right_hist_size:
                shared_memory[local_tid] = arr[256 - right_hist_size + local_tid]

            cuda.syncthreads()

        @cuda.jit(device=True)
        def pointer_jumping(cdf, jump):
            tid = cuda.threadIdx.x

            right = tid + jump
            temp = cdf[tid]
            cuda.syncthreads()
            if right < max_block_size:
                cdf[right] = scan_op(temp, cdf[right])
            cuda.syncthreads()

        @cuda.jit(device=True)
        def calculate_sum_hist(hist_temp):
            jump = 1
            while jump < cuda.blockDim.x:
                pointer_jumping(hist_temp, jump)
                jump = jump * 2

            cuda.syncthreads()

        @cuda.jit(device=True)
        def calculate_pdf_left(d_cdf_clip_hist, left_hist_size, left_hist):
            local_tid = cuda.threadIdx.x
            if local_tid < left_hist_size:
                temp = left_hist[local_tid]
                left_hist[local_tid] = temp / d_cdf_clip_hist[left_hist_size - 1]
            cuda.syncthreads()

        @cuda.jit(device=True)
        def calculate_pdf_right(d_cdf_clip_hist, right_hist_size, right_hist):
            local_tid = cuda.threadIdx.x
            if local_tid < right_hist_size:
                temp = right_hist[local_tid]
                right_hist[local_tid] = temp / (
                    d_cdf_clip_hist[-1] - d_cdf_clip_hist[256 - right_hist_size - 1]
                )
            cuda.syncthreads()

        @cuda.jit(device=True)
        def pointer_jumping(cdf, jump):
            tid = cuda.threadIdx.x

            right = tid + jump
            temp = cdf[tid]
            cuda.syncthreads()
            if right < max_block_size:
                cdf[right] = scan_op(temp, cdf[right])
            cuda.syncthreads()

        @cuda.jit(device=True)
        def calculate_cdf_hist(hist) -> None:
            jump = 1
            while jump < cuda.blockDim.x:
                pointer_jumping(hist, jump)
                jump *= 2
            cuda.syncthreads()

        def calculate_pdf_cdf(
            d_clip_hist,
            left_hist,
            right_hist,
            d_cdf_clip_hist,
            left_hist_size,
            right_hist_size,
        ):
            load_shared_memory_left_hist(left_hist, left_hist_size, d_clip_hist)
            load_shared_memory_right_hist(right_hist, right_hist_size, d_clip_hist)
            load_shared_memory_left_hist(d_cdf_clip_hist, 256, d_clip_hist)
            calculate_sum_hist(d_cdf_clip_hist)
            calculate_pdf_left(d_cdf_clip_hist, left_hist_size, left_hist)
            calculate_pdf_right(d_cdf_clip_hist, right_hist_size, right_hist)
            calculate_cdf_hist(left_hist)
            calculate_cdf_hist(right_hist)

        return cuda.jit(calculate_pdf_cdf)

    def _compile_5(self, dtype):
        key = self._functor, dtype
        if key not in self._cache_5:
            self._cache_5[key] = ExposureHistogramEqualization._gpu_kernel_factory_5(
                self._functor
            )
        return self._cache_5[key]

    @staticmethod
    def _gpu_kernel_factory_6():
        max_block_size = (
            ExposureHistogramEqualization._NUM_WARPS
            * ExposureHistogramEqualization._WARP_SIZE
        )

        @cuda.jit(device=True)
        def load_shared_memory_gray(shared_memory_gray, d_output):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)

            shared_memory_gray[local_tid] = d_output[global_tid]
            cuda.syncthreads()

        @cuda.jit(device=True)
        def calculate_esihe(shared_memory, x_mean, left_hist, right_hist):
            local_tid = cuda.threadIdx.x

            temp = shared_memory[local_tid]
            if temp < x_mean + 1:
                esihe_value = round(x_mean * left_hist[temp])
            else:
                esihe_value = round(
                    (x_mean + 1) + (255 - x_mean) * right_hist[temp - x_mean - 1]
                )

            shared_memory[local_tid] = esihe_value
            cuda.syncthreads()

        @cuda.jit(device=True)
        def map_values(shared_memory, d_output):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)

            d_output[global_tid] = shared_memory[local_tid]
            cuda.syncthreads()

        def esihe(left_hist, right_hist, d_output, x_mean):
            shared_memory_gray = cuda.shared.array(
                shape=max_block_size, dtype=np.uint32
            )
            load_shared_memory_gray(shared_memory_gray, d_output)
            calculate_esihe(shared_memory_gray, x_mean, left_hist, right_hist)
            map_values(shared_memory_gray, d_output)

        return cuda.jit(esihe)

    def _compile_6(self, dtype):
        key = dtype
        if key not in self._cache_6:
            self._cache_6[key] = ExposureHistogramEqualization._gpu_kernel_factory_6()
        return self._cache_6[key]

    @staticmethod
    def _gpu_kernel_factory_7():
        def reshape_1d_to_2d(d_input, d_output):
            global_tid = cuda.grid(1)
            width = d_output.shape[1]
            if global_tid < d_input.size:
                row = global_tid // width
                column = global_tid % width

                d_output[row, column] = d_input[global_tid]

        return cuda.jit(reshape_1d_to_2d)

    def _compile_7(self, dtype):
        key = self._functor, dtype
        if key not in self._cache_7:
            self._cache_7[key] = ExposureHistogramEqualization._gpu_kernel_factory_7()
        return self._cache_7[key]

    def __call__(self, d_input_orig):
        height, width = d_input_orig.shape[0], d_input_orig.shape[1]
        max_block_size = (
            ExposureHistogramEqualization._WARP_SIZE
            * ExposureHistogramEqualization._NUM_WARPS
        )
        stream = cuda.default_stream()

        _sav, core.config.CUDA_LOW_OCCUPANCY_WARNINGS = (
            core.config.CUDA_LOW_OCCUPANCY_WARNINGS,
            False,
        )

        d_input = cuda.device_array(
            shape=(height * width, 3), dtype=np.float32, stream=stream
        )
        d_final_output_1d = cuda.device_array(
            shape=(height * width), dtype=np.uint8, stream=stream
        )
        d_final_output_2d = cuda.device_array(
            shape=(height, width), dtype=np.uint8, stream=stream
        )

        d_hist = cuda.device_array(shape=(256,), dtype=np.float32, stream=stream)
        mean_histogram = cuda.device_array(shape=1, dtype=np.float32, stream=stream)
        sum_histogram = cuda.device_array(shape=1, dtype=np.float32, stream=stream)
        exposure = cuda.device_array(shape=1, dtype=np.float32, stream=stream)
        a_norm = cuda.device_array(shape=1, dtype=np.float32, stream=stream)
        x_mean = cuda.device_array(shape=1, dtype=np.uint8, stream=stream)
        d_clip_hist = cuda.device_array(shape=256, dtype=np.float32, stream=stream)
        d_cdf_clip_hist = cuda.device_array(shape=256, dtype=np.float32, stream=stream)

        kernel = self._compile(d_input_orig.dtype)
        kernel2 = self._compile_2(d_input_orig.dtype)
        kernel3 = self._compile_3(
            d_input_orig.dtype,
        )
        kernel4 = self._compile_4(d_input_orig.dtype)
        kernel5 = self._compile_5(d_input_orig.dtype)
        kernel6 = self._compile_6(d_input_orig.dtype)
        kernel7 = self._compile_7(d_input_orig.dtype)

        start_event = cuda.event(True)
        start_event.record(stream=stream)

        nb_threads = (16, 16)
        nb_blocks = (
            (width + nb_threads[1] - 1) // nb_threads[1],
            (height + nb_threads[0] - 1) // nb_threads[0],
        )
        kernel[nb_blocks, nb_threads, stream](d_input_orig, d_input)

        nb_threads = min(max_block_size, d_input.size)
        nb_blocks = (d_input.shape[0] + nb_threads - 1) // nb_threads
        kernel2[nb_blocks, nb_threads, stream](d_input, d_final_output_1d, d_hist)
        kernel3[1, nb_threads, stream](
            d_hist, d_clip_hist, mean_histogram, sum_histogram
        )
        kernel4[1, nb_threads, stream](
            d_hist, sum_histogram[0], exposure, a_norm, x_mean
        )
        left_hist_size = x_mean[0] + 1
        right_hist_size = 256 - left_hist_size
        left_hist = cuda.device_array(
            shape=(left_hist_size,), dtype=np.float32, stream=stream
        )
        right_hist = cuda.device_array(
            shape=(right_hist_size,), dtype=np.float32, stream=stream
        )

        kernel5[1, nb_threads, stream](
            d_clip_hist,
            left_hist,
            right_hist,
            d_cdf_clip_hist,
            left_hist_size,
            right_hist_size,
        )

        kernel6[nb_blocks, nb_threads, stream](
            left_hist, right_hist, d_final_output_1d, x_mean[0]
        )
        kernel7[nb_blocks, nb_threads, stream](d_final_output_1d, d_final_output_2d)

        stop_event = cuda.event(True)
        stop_event.record(stream=stream)
        stop_event.synchronize()

        core.config.CUDA_LOW_OCCUPANCY_WARNINGS = _sav

        return (
            cuda.event_elapsed_time(start_event, stop_event),
            d_final_output_2d.copy_to_host(),
        )


if __name__ == "__main__":

    def esihe():
        esihe = ExposureHistogramEqualization(lambda a, b: a + b)
        image_url = "https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/000cf025-48d6-45b2-987f-c19db5250699/width=1024/image%20(18).jpeg"
        response = requests.get(image_url)
        image_bytes = BytesIO(response.content)
        image = Image.open(image_bytes)
        image = np.array(image, dtype=np.float32)
        d_input_image = cuda.to_device(image)
        ct = []
        for i in range(6):
            if i == 0:
                time, result_image = esihe(d_input_image)
                result_image = Image.fromarray(result_image)
                result_image.save("esihe.png")
            else:
                time, _ = esihe(d_input_image)
            ct.append(time)
        print(f"average kernel computation time is {np.average(ct[1:])} ms")
        print(ct)

    esihe()


average kernel computation time is 0.7698495984077454 ms
[1134.9710693359375, 1.0113919973373413, 0.7313600182533264, 0.707423985004425, 0.7080000042915344, 0.6910719871520996]


### MMSICHE

In [1]:
from __future__ import annotations
import numpy as np
from numba import cuda, core
from PIL import Image
import requests
from io import BytesIO

class MedianMeanHistogramEqualization(object):
    _cache = {}
    _cache_2 = {}
    _cache_3 = {}
    _cache_4 = {}
    _cache_5 = {}
    _cache_6 = {}
    _cache_7 = {}

    _WARP_SIZE = 32
    _NUM_WARPS = 8

    def __init__(self, functor):
        self._functor = functor

    @staticmethod
    def _gpu_kernel_factory():
        def reshape_3d_to_2d(d_input_orig, d_input):
            x, y = cuda.grid(2)
            height = d_input_orig.shape[0]
            width = d_input_orig.shape[1]
            if x < width and y < height:
                idx = y * width + x
                d_input[idx][0] = d_input_orig[y, x][0]
                d_input[idx][1] = d_input_orig[y, x][1]
                d_input[idx][2] = d_input_orig[y, x][2]

        return cuda.jit(reshape_3d_to_2d)

    def _compile(self, dtype):
        key = self._functor, dtype
        if key not in self._cache:
            self._cache[key] = MedianMeanHistogramEqualization._gpu_kernel_factory()
        return self._cache[key]

    @staticmethod
    def _gpu_kernel_factory_2():
        max_block_size = (
            MedianMeanHistogramEqualization._NUM_WARPS
            * MedianMeanHistogramEqualization._WARP_SIZE
        )

        @cuda.jit(device=True)
        def load_shared_memory_input(
            shared_memory_r, shared_memory_g, shared_memory_b, d_input
        ):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)

            shared_memory_r[local_tid] = d_input[global_tid][0]
            shared_memory_g[local_tid] = d_input[global_tid][1]
            shared_memory_b[local_tid] = d_input[global_tid][2]

            cuda.syncthreads()

        @cuda.jit(device=True)
        def load_shared_memory_gray(shared_memory_gray, d_output):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)
            shared_memory_gray[local_tid] = d_output[global_tid]
            cuda.syncthreads()

        @cuda.jit(device=True)
        def grayscale_image(
            shared_memory_r, shared_memory_g, shared_memory_b, d_output
        ):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)
            grayscale_value = round(
                shared_memory_r[local_tid] * 0.299
                + shared_memory_g[local_tid] * 0.587
                + shared_memory_b[local_tid] * 0.114
            )
            d_output[global_tid] = grayscale_value

        @cuda.jit(device=True)
        def save_shared_memory_histogram(shared_memory_histogram, d_hist):
            tid = cuda.threadIdx.x
            cuda.syncthreads()
            while tid < max_block_size:
                cuda.atomic.add(d_hist, tid, shared_memory_histogram[tid])
                tid += cuda.blockDim.x

        @cuda.jit(device=True)
        def init_shared_memory_histogram(shared_memory_histogram):
            local_tid = cuda.threadIdx.x
            shared_memory_histogram[local_tid] = 0
            cuda.syncthreads()

        def histogram(d_input, d_output, d_hist):
            shared_memory_red = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )
            shared_memory_green = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )
            shared_memory_blue = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )
            shared_memory_gray = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )

            load_shared_memory_input(
                shared_memory_red, shared_memory_green, shared_memory_blue, d_input
            )

            grayscale_image(
                shared_memory_red, shared_memory_green, shared_memory_blue, d_output
            )

            shared_memory_histogram = cuda.shared.array(shape=256, dtype=np.float32)

            load_shared_memory_gray(shared_memory_gray, d_output)
            init_shared_memory_histogram(shared_memory_histogram)
            tid = cuda.grid(1)
            gridDim = cuda.gridDim.x * cuda.blockDim.x
            while tid < d_output.size:
                pixel_values = d_output[tid]
                cuda.atomic.add(shared_memory_histogram, pixel_values, 1)
                tid += gridDim
            save_shared_memory_histogram(shared_memory_histogram, d_hist)

        return cuda.jit(histogram)

    def _compile_2(self, dtype):
        key = dtype
        if key not in self._cache_2:
            self._cache_2[key] = MedianMeanHistogramEqualization._gpu_kernel_factory_2()
        return self._cache_2[key]

    @staticmethod
    def _gpu_kernel_factory_3(fn):
        scan_op = cuda.jit(device=True)(fn)
        max_block_size = (
            MedianMeanHistogramEqualization._NUM_WARPS
            * MedianMeanHistogramEqualization._WARP_SIZE
        )

        @cuda.jit(device=True)
        def load_shared_memory(shared_memory, arr):
            local_tid = cuda.threadIdx.x
            if local_tid < arr.size:
                shared_memory[local_tid] = arr[local_tid]

        @cuda.jit(device=True)
        def pointer_jumping(cdf, jump):
            tid = cuda.threadIdx.x

            right = tid + jump
            temp = cdf[tid]
            cuda.syncthreads()
            if right < max_block_size:
                cdf[right] = scan_op(temp, cdf[right])
            cuda.syncthreads()

        @cuda.jit(device=True)
        def calculate_cdf_histogram(d_hist, partials):
            load_shared_memory(partials, d_hist)

            jump = 1
            while jump < cuda.blockDim.x:
                pointer_jumping(partials, jump)
                jump = jump * 2

        @cuda.jit(device=True)
        def calculate_median_histogram(partials, median_histogram):
            local_tid = cuda.threadIdx.x
            sum_histogram = partials[-1]
            if (
                local_tid > 0
                and partials[local_tid] >= sum_histogram / 2
                and partials[local_tid - 1] < sum_histogram / 2
            ):
                median_histogram[0] = local_tid

            cuda.syncthreads()

        @cuda.jit(device=True)
        def clip_histogram(d_hist, d_clip_hist, clip_value):
            local_tid = cuda.threadIdx.x
            if local_tid < max_block_size:
                temp = d_hist[local_tid]
                if temp > clip_value:
                    d_clip_hist[local_tid] = clip_value
                else:
                    d_clip_hist[local_tid] = temp

        def create_clip_histogram(d_hist, d_clip_hist, median_histogram):
            partials = cuda.shared.array(shape=256, dtype=np.int32)
            calculate_cdf_histogram(d_hist, partials)
            calculate_median_histogram(partials, median_histogram)
            clip_histogram(d_hist, d_clip_hist, d_hist[median_histogram[0]])

        return cuda.jit(create_clip_histogram)

    def _compile_3(self, dtype):
        key = self._functor, dtype
        if key not in self._cache_3:
            self._cache_3[key] = MedianMeanHistogramEqualization._gpu_kernel_factory_3(
                self._functor
            )
        return self._cache_3[key]

    @staticmethod
    def _gpu_kernel_factory_4(fn):
        scan_op = cuda.jit(device=True)(fn)

        @cuda.jit(device=True)
        def load_shared_memory_left_hist(shared_memory, left_hist_size, arr):
            local_tid = cuda.threadIdx.x
            if local_tid < left_hist_size:
                shared_memory[local_tid] = arr[local_tid]
            else:
                shared_memory[local_tid] = 0
            cuda.syncthreads()

        @cuda.jit(device=True)
        def load_shared_memory_right_hist(shared_memory, right_hist_size, arr):
            local_tid = cuda.threadIdx.x
            if local_tid < right_hist_size:
                shared_memory[local_tid] = arr[256 - right_hist_size + local_tid]
            else:
                shared_memory[local_tid] = 0
            cuda.syncthreads()

        @cuda.jit(device=True)
        def pointer_jumping(cdf, jump, hist_size):
            tid = cuda.threadIdx.x

            right = tid + jump
            temp = cdf[tid]
            cuda.syncthreads()
            if right < hist_size:
                cdf[right] = scan_op(temp, cdf[right])
            cuda.syncthreads()

        @cuda.jit(device=True)
        def calculate_sum_hist(hist_temp, hist_size):
            jump = 1
            while jump < hist_size:
                pointer_jumping(hist_temp, jump, hist_size)
                jump = jump * 2

            cuda.syncthreads()

        @cuda.jit(device=True)
        def calculate_pdf_hist(hist_temp, hist_size, hist):
            local_tid = cuda.threadIdx.x
            if local_tid < hist_size:
                temp = hist[local_tid]
                hist[local_tid] = temp / hist_temp[-1]
            cuda.syncthreads()

        @cuda.jit(device=True)
        def multiply_left_index(hist, left_hist_size):
            local_tid = cuda.threadIdx.x
            if local_tid < left_hist_size:
                temp = hist[local_tid]
                hist[local_tid] = temp * local_tid

            cuda.syncthreads()

        @cuda.jit(device=True)
        def multiply_right_index(hist, right_hist_size):
            local_tid = cuda.threadIdx.x
            if local_tid < right_hist_size:
                temp = hist[local_tid]
                hist[local_tid] = temp * (local_tid + 256 - right_hist_size)

            cuda.syncthreads()

        def calculate_pdf_cdf(
            d_hist,
            left_hist,
            right_hist,
            left_hist_temp,
            right_hist_temp,
            left_hist_size,
            right_hist_size,
            x_ml,
            x_mu,
        ):
            load_shared_memory_left_hist(left_hist, left_hist_size, d_hist)

            load_shared_memory_right_hist(right_hist, right_hist_size, d_hist)
            load_shared_memory_left_hist(left_hist_temp, left_hist_size, d_hist)
            load_shared_memory_right_hist(right_hist_temp, right_hist_size, d_hist)
            calculate_sum_hist(left_hist_temp, left_hist_size)
            cuda.syncthreads()

            calculate_sum_hist(right_hist_temp, right_hist_size)

            calculate_pdf_hist(left_hist_temp, left_hist_size, left_hist)
            calculate_pdf_hist(right_hist_temp, right_hist_size, right_hist)
            multiply_left_index(left_hist, left_hist_size)
            multiply_right_index(right_hist, right_hist_size)

            calculate_sum_hist(left_hist, left_hist_size)
            calculate_sum_hist(right_hist, right_hist_size)

            if cuda.threadIdx.x == 0:
                x_ml[0] = round(left_hist[-1])
                x_mu[0] = round(right_hist[-1])

        return cuda.jit(calculate_pdf_cdf)

    def _compile_4(self, dtype):
        key = self._functor, dtype
        if key not in self._cache_4:
            self._cache_4[key] = MedianMeanHistogramEqualization._gpu_kernel_factory_4(
                self._functor
            )
        return self._cache_4[key]

    @staticmethod
    def _gpu_kernel_factory_5(fn):
        scan_op = cuda.jit(device=True)(fn)

        @cuda.jit(device=True)
        def load_hist(
            shared_memory_hist_clip,
            d_clip_hist,
            first_hist,
            second_hist,
            third_hist,
            fourth_hist,
            left_index,
            median,
            right_index,
        ):
            local_tid = cuda.threadIdx.x

            if local_tid <= left_index:
                num_pixels = d_clip_hist[left_index]
                first_hist[local_tid] = shared_memory_hist_clip[local_tid] / num_pixels
            elif local_tid > left_index and local_tid <= median:
                num_pixels = d_clip_hist[median] - d_clip_hist[left_index]
                second_hist[local_tid - (left_index + 1)] = (
                    shared_memory_hist_clip[local_tid] / num_pixels
                )

            elif local_tid > median and local_tid <= right_index:
                num_pixels = d_clip_hist[right_index] - d_clip_hist[median]
                third_hist[local_tid - (median + 1)] = (
                    shared_memory_hist_clip[local_tid] / num_pixels
                )

            else:
                num_pixels = d_clip_hist[-1] - d_clip_hist[right_index]
                fourth_hist[local_tid - (right_index + 1)] = (
                    shared_memory_hist_clip[local_tid] / num_pixels
                )

            cuda.syncthreads()

        @cuda.jit(device=True)
        def pointer_jumping(cdf, jump, hist_size):
            tid = cuda.threadIdx.x

            right = tid + jump
            temp = cdf[tid]
            cuda.syncthreads()
            if right < hist_size:
                cdf[right] = scan_op(temp, cdf[right])
            cuda.syncthreads()

        @cuda.jit(device=True)
        def calculate_cdf_hist(hist, hist_size) -> None:
            jump = 1
            while jump < hist_size:
                pointer_jumping(hist, jump, hist_size)
                jump *= 2
            cuda.syncthreads()

        @cuda.jit(device=True)
        def load_shared_memory_hist(shared_memory_hist_clip, d_clip_hist):
            local_tid = cuda.threadIdx.x

            shared_memory_hist_clip[local_tid] = d_clip_hist[local_tid]
            cuda.syncthreads()

        def calculate_pdf_cdf(
            d_clip_hist,
            first_hist,
            second_hist,
            third_hist,
            fourth_hist,
            x_ml,
            x_mu,
            median_value,
        ):
            shared_memory_hist_clip = cuda.shared.array(shape=256, dtype=np.float32)
            load_shared_memory_hist(shared_memory_hist_clip, d_clip_hist)
            calculate_cdf_hist(d_clip_hist, 256)
            load_hist(
                shared_memory_hist_clip,
                d_clip_hist,
                first_hist,
                second_hist,
                third_hist,
                fourth_hist,
                x_ml,
                median_value,
                x_mu,
            )

            calculate_cdf_hist(first_hist, x_ml + 1)
            calculate_cdf_hist(second_hist, median_value - x_ml)
            calculate_cdf_hist(third_hist, x_mu - median_value)
            calculate_cdf_hist(fourth_hist, 255 - x_mu)

        return cuda.jit(calculate_pdf_cdf)

    def _compile_5(self, dtype):
        key = self._functor, dtype
        if key not in self._cache_5:
            self._cache_5[key] = MedianMeanHistogramEqualization._gpu_kernel_factory_5(
                self._functor
            )
        return self._cache_5[key]

    @staticmethod
    def _gpu_kernel_factory_6():
        max_block_size = (
            MedianMeanHistogramEqualization._NUM_WARPS
            * MedianMeanHistogramEqualization._WARP_SIZE
        )

        @cuda.jit(device=True)
        def load_shared_memory(shared_memory, d_output):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)

            shared_memory[local_tid] = d_output[global_tid]
            cuda.syncthreads()

        @cuda.jit(device=True)
        def calculate_mmsiche(
            shared_memory,
            first_hist,
            second_hist,
            third_hist,
            fourth_hist,
            x_ml,
            x_mu,
            median_value,
        ):
            local_tid = cuda.threadIdx.x

            temp = shared_memory[local_tid]
            if temp <= x_ml:
                esihe_value = x_ml * first_hist[temp]
            elif temp > x_ml and temp <= median_value:
                esihe_value = (x_ml + 1) + second_hist[temp - x_ml - 1] * (
                    median_value - x_ml + 1
                )
            elif temp > median_value and temp <= x_mu:
                esihe_value = (median_value + 1) + third_hist[
                    temp - median_value - 1
                ] * (x_mu - median_value + 1)
            elif temp > x_mu and temp < 256:
                esihe_value = (x_mu + 1) + fourth_hist[temp - x_mu - 1] * (
                    256 - x_mu + 1
                )
            shared_memory[local_tid] = esihe_value
            cuda.syncthreads()

        @cuda.jit(device=True)
        def map_values(shared_memory, d_output):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)

            d_output[global_tid] = shared_memory[local_tid]
            cuda.syncthreads()

        def mmsiche(
            first_hist,
            second_hist,
            third_hist,
            fourth_hist,
            x_ml,
            x_mu,
            median_value,
            d_output,
        ):
            shared_memory = cuda.shared.array(shape=max_block_size, dtype=np.uint32)
            load_shared_memory(shared_memory, d_output)
            calculate_mmsiche(
                shared_memory,
                first_hist,
                second_hist,
                third_hist,
                fourth_hist,
                x_ml,
                x_mu,
                median_value,
            )
            map_values(shared_memory, d_output)

        return cuda.jit(mmsiche)

    def _compile_6(self, dtype):
        key = dtype
        if key not in self._cache_6:
            self._cache_6[key] = MedianMeanHistogramEqualization._gpu_kernel_factory_6()
        return self._cache_6[key]

    @staticmethod
    def _gpu_kernel_factory_7():
        def reshape_1d_to_2d(d_input, d_output):
            global_tid = cuda.grid(1)
            width = d_output.shape[1]
            if global_tid < d_input.size:
                row = global_tid // width
                column = global_tid % width

                d_output[row, column] = d_input[global_tid]

        return cuda.jit(reshape_1d_to_2d)

    def _compile_7(self, dtype):
        key = self._functor, dtype
        if key not in self._cache_7:
            self._cache_7[key] = MedianMeanHistogramEqualization._gpu_kernel_factory_7()
        return self._cache_7[key]

    def __call__(self, d_input_orig):
        height, width = d_input_orig.shape[0], d_input_orig.shape[1]
        max_block_size = (
            MedianMeanHistogramEqualization._WARP_SIZE
            * MedianMeanHistogramEqualization._NUM_WARPS
        )
        stream = cuda.default_stream()

        _sav, core.config.CUDA_LOW_OCCUPANCY_WARNINGS = (
            core.config.CUDA_LOW_OCCUPANCY_WARNINGS,
            False,
        )

        d_input = cuda.device_array(
            shape=(height * width, 3), dtype=np.float32, stream=stream
        )
        d_final_output_1d = cuda.device_array(
            shape=(height * width), dtype=np.uint8, stream=stream
        )
        d_final_output_2d = cuda.device_array(
            shape=(height, width), dtype=np.uint8, stream=stream
        )
        median_histogram = cuda.device_array(shape=1, dtype=np.uint32, stream=stream)
        d_hist = cuda.device_array(shape=(256,), dtype=np.float32, stream=stream)
        d_clip_hist = cuda.device_array(shape=256, dtype=np.float32, stream=stream)
        x_ml = cuda.device_array(shape=1, dtype=np.uint32, stream=stream)
        x_mu = cuda.device_array(shape=1, dtype=np.uint32, stream=stream)

        kernel = self._compile(d_input_orig.dtype)
        kernel2 = self._compile_2(d_input_orig.dtype)
        kernel3 = self._compile_3(
            d_input_orig.dtype,
        )
        kernel4 = self._compile_4(d_input_orig.dtype)
        kernel5 = self._compile_5(d_input_orig.dtype)
        kernel6 = self._compile_6(d_input_orig.dtype)
        kernel7 = self._compile_7(d_input_orig.dtype)

        start_event = cuda.event(True)
        start_event.record(stream=stream)
        nb_threads = max_block_size

        nb_threads = (16, 16)
        nb_blocks = (
            (width + nb_threads[1] - 1) // nb_threads[1],
            (height + nb_threads[0] - 1) // nb_threads[0],
        )
        kernel[nb_blocks, nb_threads, stream](d_input_orig, d_input)

        nb_threads = min(max_block_size, d_input.size)
        nb_blocks = (d_input.shape[0] + nb_threads - 1) // nb_threads

        kernel2[nb_blocks, nb_threads, stream](d_input, d_final_output_1d, d_hist)
        kernel3[1, nb_threads, stream](d_hist, d_clip_hist, median_histogram)

        left_hist_size = median_histogram[0]
        right_hist_size = 256 - left_hist_size
        left_hist = cuda.device_array(
            shape=(left_hist_size,), dtype=np.float32, stream=stream
        )
        right_hist = cuda.device_array(
            shape=(right_hist_size,), dtype=np.float32, stream=stream
        )

        left_hist_temp = cuda.device_array(
            shape=(left_hist_size,), dtype=np.float32, stream=stream
        )
        right_hist_temp = cuda.device_array(
            shape=(right_hist_size,), dtype=np.float32, stream=stream
        )
        kernel4[1, nb_threads, stream](
            d_hist,
            left_hist,
            right_hist,
            left_hist_temp,
            right_hist_temp,
            left_hist_size,
            right_hist_size,
            x_ml,
            x_mu,
        )

        first_hist_size = x_ml[0] + 1
        first_hist = cuda.device_array(
            shape=(first_hist_size,), dtype=np.float32, stream=stream
        )
        second_hist_size = median_histogram[0] - x_ml[0]
        second_hist = cuda.device_array(
            shape=(second_hist_size,), dtype=np.float32, stream=stream
        )
        third_hist_size = x_mu[0] - median_histogram[0]
        third_hist = cuda.device_array(
            shape=(third_hist_size,), dtype=np.float32, stream=stream
        )
        fourth_hist_size = 255 - x_mu[0]
        fourth_hist = cuda.device_array(
            shape=(fourth_hist_size,), dtype=np.float32, stream=stream
        )
        kernel5[1, nb_threads, stream](
            d_clip_hist,
            first_hist,
            second_hist,
            third_hist,
            fourth_hist,
            x_ml[0],
            x_mu[0],
            median_histogram[0],
        )

        kernel6[nb_blocks, nb_threads, stream](
            first_hist,
            second_hist,
            third_hist,
            fourth_hist,
            x_ml[0],
            x_mu[0],
            median_histogram[0],
            d_final_output_1d,
        )
        kernel7[nb_blocks, nb_threads, stream](d_final_output_1d, d_final_output_2d)

        stop_event = cuda.event(True)
        stop_event.record(stream=stream)
        stop_event.synchronize()

        core.config.CUDA_LOW_OCCUPANCY_WARNINGS = _sav

        return (
            cuda.event_elapsed_time(start_event, stop_event),
            d_final_output_2d.copy_to_host(),
        )


if __name__ == "__main__":

    def mmsiche():
        mmsiche = MedianMeanHistogramEqualization(lambda a, b: a + b)
        image_url = "https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/000cf025-48d6-45b2-987f-c19db5250699/width=1024/image%20(18).jpeg"
        response = requests.get(image_url)
        image_bytes = BytesIO(response.content)
        image = Image.open(image_bytes)
        image = np.array(image, dtype=np.float32)
        d_input_image = cuda.to_device(image)

        ct = []
        for i in range(6):
            if i == 0:
                time, result_image = mmsiche(d_input_image)
                result_image = Image.fromarray(result_image)
                result_image.save("mmsiche.png")

            else:
                time, _ = mmsiche(d_input_image)
            ct.append(time)
        print(f"average kernel computation time is {np.average(ct[1:])} ms")
        print(ct)

    mmsiche()


average kernel computation time is 1.7328320026397706 ms
[1328.797119140625, 1.8748480081558228, 1.6861439943313599, 1.7666239738464355, 1.7007360458374023, 1.635807991027832]


### CLAHE

In [1]:
from __future__ import annotations
import numpy as np
from numba import cuda, core
from PIL import Image
import requests
from io import BytesIO
import math

class CLAHE(object):
    _cache = {}
    _cache_2 = {}

    def __init__(self, functor):
        self._functor = functor

    @staticmethod
    def _gpu_kernel_factory(fn):
        scan_op = cuda.jit(device=True)(fn)

        @cuda.jit(device=True)
        def load_shared_memory_input(shared_memory_input, d_input):
            local_tid_x = cuda.threadIdx.x
            local_tid_y = cuda.threadIdx.y
            x, y = cuda.grid(2)
            if y < d_input.shape[0] and x < d_input.shape[1]:
                shared_memory_input[local_tid_y, local_tid_x, 0] = d_input[y, x, 0]
                shared_memory_input[local_tid_y, local_tid_x, 1] = d_input[y, x, 1]
                shared_memory_input[local_tid_y, local_tid_x, 2] = d_input[y, x, 2]

            cuda.syncthreads()

        @cuda.jit(device=True)
        def load_shared_memory_gray(shared_memory_gray, d_output):
            local_tid_x = cuda.threadIdx.x
            local_tid_y = cuda.threadIdx.y
            x, y = cuda.grid(2)
            if y < d_output.shape[0] and x < d_output.shape[1]:
                shared_memory_gray[local_tid_y, local_tid_x] = d_output[y, x]
            cuda.syncthreads()

        @cuda.jit(device=True)
        def grayscale_image(shared_memory_input, d_output):
            local_tid_x = cuda.threadIdx.x
            local_tid_y = cuda.threadIdx.y
            x, y = cuda.grid(2)
            d_output[y, x] = round(
                shared_memory_input[local_tid_y, local_tid_x, 0] * 0.299
                + shared_memory_input[local_tid_y, local_tid_x, 1] * 0.587
                + shared_memory_input[local_tid_y, local_tid_x, 2] * 0.114
            )
            cuda.syncthreads()

        @cuda.jit(device=True)
        def calculate_histogram(shared_memory_gray, shared_memory_hist):
            local_tid_x = cuda.threadIdx.x
            local_tid_y = cuda.threadIdx.y
            temp = shared_memory_gray[local_tid_y, local_tid_x]
            cuda.atomic.add(shared_memory_hist, temp, 1)
            cuda.syncthreads()

        @cuda.jit(device=True)
        def load_shared_memory_exceed(
            shared_memory_hist, shared_memory_exceed, clip_limit
        ):
            local_tid_x = cuda.threadIdx.x
            local_tid_y = cuda.threadIdx.y
            value = shared_memory_hist[local_tid_x * 16 + local_tid_y]
            if value > clip_limit:
                shared_memory_exceed[local_tid_x * 16 + local_tid_y] = (
                    value - clip_limit
                )
                shared_memory_hist[local_tid_x * 16 + local_tid_y] = clip_limit
            cuda.syncthreads()

        @cuda.jit(device=True)
        def calculate_cdf(shared_memory_hist) -> None:
            local_tid_x = cuda.threadIdx.x
            local_tid_y = cuda.threadIdx.y
            jump = 1
            while jump < 256:
                tid = local_tid_x * 16 + local_tid_y
                right = tid + jump
                temp = shared_memory_hist[tid]
                cuda.syncthreads()
                if right < 256:
                    shared_memory_hist[right] = scan_op(temp, shared_memory_hist[right])
                cuda.syncthreads()
                jump *= 2
            cuda.syncthreads()

        @cuda.jit(device=True)
        def redistribute(shared_memory_hist, shared_memory_exceed, clip_limit):
            local_tid_x = cuda.threadIdx.x
            local_tid_y = cuda.threadIdx.y
            if local_tid_x == 0 and local_tid_y == 0:
                total_exceed = shared_memory_exceed
                bin_incr = total_exceed / 256
                upper = clip_limit - bin_incr
                for i in range(256):
                    if shared_memory_hist[i] > clip_limit:
                        shared_memory_hist[i] = clip_limit
                    else:
                        if shared_memory_hist[i] > upper:
                            total_exceed += upper - shared_memory_hist[i]
                            shared_memory_hist[i] = clip_limit
                        else:
                            total_exceed -= bin_incr
                            shared_memory_hist[i] += bin_incr
                    if total_exceed < 1:
                        break

                if total_exceed > 0:
                    step_size = max(1, math.floor(1 + total_exceed / 256))
                    for i in range(256):
                        total_exceed -= step_size
                        shared_memory_hist[i] += step_size
                        if total_exceed < 1:
                            break

            else:
                return

        @cuda.jit(device=True)
        def save_cdf(cdf_all, shared_memory_hist):
            local_tid_x = cuda.threadIdx.x
            local_tid_y = cuda.threadIdx.y
            local_block_x = cuda.blockIdx.x
            local_block_y = cuda.blockIdx.y
            tid = local_tid_y * 16 + local_tid_x
            cdf_all[local_block_y, local_block_x, tid] = shared_memory_hist[tid]

        @cuda.jit(device=True)
        def init_shared_memory(shared_memory_hist):
            local_tid_x = cuda.threadIdx.x
            local_tid_y = cuda.threadIdx.y
            shared_memory_hist[local_tid_x * 16 + local_tid_y] = 0
            cuda.syncthreads()

        @cuda.jit(device=True)
        def normalized_cdf(shared_memory_hist, max_value):
            local_tid_x = cuda.threadIdx.x
            local_tid_y = cuda.threadIdx.y
            tid = local_tid_x * 16 + local_tid_y
            if max_value > 0 and tid < 256:
                temp = shared_memory_hist[tid]
                shared_memory_hist[tid] = (temp * 255) // max_value

        def histogram(d_input, d_output, clip_limit, cdf_all):
            shared_memory_input = cuda.shared.array(shape=(16, 16, 3), dtype=np.float32)
            shared_memory_gray = cuda.shared.array(shape=(16, 16), dtype=np.float32)
            shared_memory_hist = cuda.shared.array(shape=256, dtype=np.float32)
            shared_memory_exceed = cuda.shared.array(shape=256, dtype=np.float32)
            load_shared_memory_input(shared_memory_input, d_input)
            grayscale_image(shared_memory_input, d_output)
            init_shared_memory(shared_memory_hist)
            init_shared_memory(shared_memory_exceed)
            load_shared_memory_gray(shared_memory_gray, d_output)
            calculate_histogram(shared_memory_gray, shared_memory_hist)
            load_shared_memory_exceed(
                shared_memory_hist, shared_memory_exceed, clip_limit
            )
            calculate_cdf(shared_memory_exceed)
            redistribute(shared_memory_hist, shared_memory_exceed[-1], clip_limit)
            cuda.syncthreads()
            calculate_cdf(shared_memory_hist)
            normalized_cdf(shared_memory_hist, shared_memory_hist[-1])
            cuda.syncthreads()
            save_cdf(cdf_all, shared_memory_hist)

        return cuda.jit(histogram)

    def _compile(self, dtype):
        key = self._functor, dtype
        if key not in self._cache:
            self._cache[key] = CLAHE._gpu_kernel_factory(self._functor)
        return self._cache[key]

    @staticmethod
    def _gpu_kernel_factory_2():
        @cuda.jit(device=True)
        def interpolate(d_output, cdf_all):
            x, y = cuda.grid(2)
            height = d_output.shape[0]
            width = d_output.shape[1]

            x_dim = cuda.blockDim.x
            y_dim = cuda.blockDim.y
            local_tid_x = cuda.threadIdx.x
            local_tid_y = cuda.threadIdx.y
            num = x_dim * y_dim
            if (
                y >= y_dim / 2
                and x >= x_dim / 2
                and y < height - y_dim / 2
                and x < width - x_dim / 2
            ):
                block_x = (x - 8) // x_dim + 1
                block_y = (y - 8) // y_dim + 1
                fourth_block_x, fourth_block_y = block_x, block_y
                second_block_x, second_block_y = block_x - 1, block_y
                third_block_x, third_block_y = block_x, block_y - 1
                first_block_x, first_block_y = block_x - 1, block_y - 1

                inverse_i = x_dim - ((x - 8) % x_dim)
                inverse_j = y_dim - ((y - 8) % y_dim)
                val = d_output[y, x]
                if local_tid_x >= x_dim / 2 and local_tid_x < x_dim:
                    j = local_tid_x - x_dim / 2
                else:
                    j = local_tid_x + x_dim / 2

                if local_tid_y >= y_dim / 2 and local_tid_y < y_dim:
                    i = local_tid_y - y_dim / 2
                else:
                    i = local_tid_y + y_dim / 2
                d_output[y, x] = math.floor(
                    (
                        inverse_j
                        * (
                            inverse_i * cdf_all[first_block_y, first_block_x, val]
                            + j * cdf_all[third_block_y, third_block_x, val]
                        )
                        + i
                        * (
                            inverse_i * cdf_all[second_block_y, second_block_x, val]
                            + j * cdf_all[fourth_block_y, fourth_block_x, val]
                        )
                    )
                    / num
                )
            elif y < 8 and x < 8:
                block_x = 0
                block_y = 0
                inverse_i = x_dim - x
                inverse_j = y_dim - y
                val = d_output[y, x]
                j = local_tid_x
                i = local_tid_y
                d_output[y, x] = math.floor(
                    (
                        inverse_j
                        * (
                            inverse_i * cdf_all[block_y, block_x, val]
                            + j * cdf_all[block_y, block_x, val]
                        )
                        + i
                        * (
                            inverse_i * cdf_all[block_y, block_x, val]
                            + j * cdf_all[block_y, block_x, val]
                        )
                    )
                    / num
                )
            elif y >= height - y_dim / 2 and x >= width - x_dim / 2:
                block_x = (x - 8) // x_dim
                block_y = (y - 8) // y_dim
                inverse_i = x_dim - (x % 8)
                inverse_j = y_dim - (y % 8)
                val = d_output[y, x]
                j = local_tid_x - x_dim / 2
                i = local_tid_y - y_dim / 2
                d_output[y, x] = math.floor(
                    (
                        inverse_j
                        * (
                            inverse_i * cdf_all[block_y, block_x, val]
                            + j * cdf_all[block_y, block_x, val]
                        )
                        + i
                        * (
                            inverse_i * cdf_all[block_y, block_x, val]
                            + j * cdf_all[block_y, block_x, val]
                        )
                    )
                    / num
                )
            elif x < 8 and y >= height - y_dim / 2:
                block_x = 0
                block_y = (y - 8) // y_dim
                inverse_i = x_dim - x
                inverse_j = y_dim - (y % 8)
                val = d_output[y, x]
                j = local_tid_x
                i = local_tid_y - y_dim / 2
                d_output[y, x] = math.floor(
                    (
                        inverse_j
                        * (
                            inverse_i * cdf_all[block_y, block_x, val]
                            + j * cdf_all[block_y, block_x, val]
                        )
                        + i
                        * (
                            inverse_i * cdf_all[block_y, block_x, val]
                            + j * cdf_all[block_y, block_x, val]
                        )
                    )
                    / num
                )
            elif y < 8 and x >= width - x_dim / 2:
                block_x = (x - 8) // x_dim
                block_y = 0
                inverse_i = x_dim - (x % 8)
                inverse_j = y_dim - y
                val = d_output[y, x]
                i = local_tid_y
                j = local_tid_x - x_dim / 2
                d_output[y, x] = math.floor(
                    (
                        inverse_j
                        * (
                            inverse_i * cdf_all[block_y, block_x, val]
                            + j * cdf_all[block_y, block_x, val]
                        )
                        + i
                        * (
                            inverse_i * cdf_all[block_y, block_x, val]
                            + j * cdf_all[block_y, block_x, val]
                        )
                    )
                    / num
                )
            elif y >= 8 and y < height - y_dim / 2 and x < 8:
                block_x = 0
                first_block_y = (y - 8) // y_dim + 1
                second_block_y = first_block_y - 1
                inverse_i = x_dim - x
                inverse_j = y_dim - ((y - 8) % y_dim)
                val = d_output[y, x]
                j = local_tid_x
                if local_tid_y >= y_dim / 2 and local_tid_y < y_dim:
                    i = local_tid_y - y_dim / 2
                else:
                    i = local_tid_y + y_dim / 2
                d_output[y, x] = math.floor(
                    (
                        inverse_j
                        * (
                            inverse_i * cdf_all[first_block_y, block_x, val]
                            + j * cdf_all[first_block_y, block_x, val]
                        )
                        + i
                        * (
                            inverse_i * cdf_all[second_block_y, block_x, val]
                            + j * cdf_all[second_block_y, block_x, val]
                        )
                    )
                    / num
                )

            elif x >= 8 and x < width - x_dim / 2 and y < 8:
                block_y = 0
                first_block_x = (x - 8) // x_dim + 1
                second_block_x = first_block_x - 1
                inverse_i = x_dim - ((x - 8) % x_dim)
                inverse_j = y_dim - y

                val = d_output[y, x]
                i = local_tid_y
                if local_tid_x >= x_dim / 2 and local_tid_x < x_dim:
                    j = local_tid_x - x_dim / 2
                else:
                    j = local_tid_x + x_dim / 2
                d_output[y, x] = math.floor(
                    (
                        inverse_j
                        * (
                            inverse_i * cdf_all[block_y, first_block_x, val]
                            + j * cdf_all[block_y, first_block_x, val]
                        )
                        + i
                        * (
                            inverse_i * cdf_all[block_y, second_block_x, val]
                            + j * cdf_all[block_y, second_block_x, val]
                        )
                    )
                    / num
                )

            elif x >= 8 and x < width - x_dim / 2 and y >= height - y_dim / 2:
                block_y = (y - 8) // y_dim
                first_block_x = (x - 8) // x_dim + 1
                second_block_x = first_block_x - 1
                inverse_i = x_dim - ((x - 8) % x_dim)
                inverse_j = y_dim - (y % 8)

                val = d_output[y, x]
                i = local_tid_y - y_dim / 2
                if local_tid_x >= x_dim / 2 and local_tid_x < x_dim:
                    j = local_tid_x - x_dim / 2
                else:
                    j = local_tid_x + x_dim / 2

                d_output[y, x] = math.floor(
                    (
                        inverse_j
                        * (
                            inverse_i * cdf_all[block_y, first_block_x, val]
                            + j * cdf_all[block_y, first_block_x, val]
                        )
                        + i
                        * (
                            inverse_i * cdf_all[block_y, second_block_x, val]
                            + j * cdf_all[block_y, second_block_x, val]
                        )
                    )
                    / num
                )

            elif y >= 8 and y < height - y_dim / 2 and x >= width - x_dim / 2:
                block_x = (x - 8) // x_dim
                first_block_y = (y - 8) // y_dim + 1
                second_block_y = first_block_y - 1
                inverse_j = y_dim - ((y - 8) % y_dim)
                inverse_i = x_dim - (x % 8)

                val = d_output[y, x]
                j = local_tid_x - x_dim / 2
                if local_tid_y >= y_dim / 2 and local_tid_y < y_dim:
                    i = local_tid_y - y_dim / 2
                else:
                    i = local_tid_y + y_dim / 2

                d_output[y, x] = math.floor(
                    (
                        inverse_j
                        * (
                            inverse_i * cdf_all[first_block_y, block_x, val]
                            + j * cdf_all[first_block_y, block_x, val]
                        )
                        + i
                        * (
                            inverse_i * cdf_all[second_block_y, block_x, val]
                            + j * cdf_all[second_block_y, block_x, val]
                        )
                    )
                    / num
                )
            cuda.syncthreads()

        def clahe(d_output, cdf_all):
            interpolate(d_output, cdf_all)

        return cuda.jit(clahe)

    def _compile_2(self, dtype):
        key = dtype
        if key not in self._cache_2:
            self._cache_2[key] = CLAHE._gpu_kernel_factory_2()
        return self._cache_2[key]

    def __call__(self, d_input_orig, clip_limit):
        height, width = d_input_orig.shape[0], d_input_orig.shape[1]
        stream = cuda.default_stream()
        nb_threads = (16, 16)
        nb_blocks = (
            (d_input_orig.shape[1] + nb_threads[1] - 1) // nb_threads[1],
            (d_input_orig.shape[0] + nb_threads[0] - 1) // nb_threads[0],
        )
        _sav, core.config.CUDA_LOW_OCCUPANCY_WARNINGS = (
            core.config.CUDA_LOW_OCCUPANCY_WARNINGS,
            False,
        )

        cdf_all = cuda.device_array((nb_blocks[1], nb_blocks[0], 256), dtype=np.float32)
        d_output = cuda.device_array((height, width), dtype=np.uint8)
        kernel = self._compile(d_input_orig.dtype)
        kernel2 = self._compile_2(d_input_orig.dtype)

        start_event = cuda.event(True)
        start_event.record(stream=stream)

        kernel[nb_blocks, nb_threads](d_input_orig, d_output, clip_limit, cdf_all)
        kernel2[nb_blocks, nb_threads](d_output, cdf_all)

        stop_event = cuda.event(True)
        stop_event.record(stream=stream)
        stop_event.synchronize()

        core.config.CUDA_LOW_OCCUPANCY_WARNINGS = _sav

        return cuda.event_elapsed_time(start_event, stop_event), d_output.copy_to_host()


if __name__ == "__main__":

    def clahe():
        clahe = CLAHE(lambda a, b: a + b)
        image_url = "https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/000cf025-48d6-45b2-987f-c19db5250699/width=1024/image%20(18).jpeg"
        response = requests.get(image_url)
        image_bytes = BytesIO(response.content)
        image = Image.open(image_bytes)
        image = np.array(image, dtype=np.float32)
        d_input_image = cuda.to_device(image)
        clip_limit = 6
        ct = []

        for i in range(6):
            if i == 0:
                time, result_image = clahe(d_input_image, clip_limit)
                result_image = Image.fromarray(result_image)
                result_image.save("clahe.png")
            else:
                time, _ = clahe(d_input_image, clip_limit)

            ct.append(time)
        print(f"average kernel computation time is {np.average(ct[1:])} ms")
        print(ct)

    clahe()


average kernel computation time is 2.1190207958221436 ms
[993.296630859375, 2.1437759399414062, 2.1122241020202637, 2.110527992248535, 2.1091840267181396, 2.119391918182373]


### Metrics & Evaluation

### AME

In [1]:
from __future__ import annotations
import numpy as np
from numba import cuda, core
from PIL import Image
import requests
from io import BytesIO


class AMEReduce(object):
    _cache = {}
    _cache_2 = {}
    _cache_3 = {}
    _cache_4 = {}
    _cache_5 = {}
    _WARP_SIZE = 32
    _NUM_WARPS = 8

    def __init__(self, functor, functor2):
        self._functor = functor
        self._functor2 = functor2

    @staticmethod
    def _gpu_kernel_factory():
        def reshape_3d_to_2d(d_input_orig, d_input):
            x, y = cuda.grid(2)
            height = d_input_orig.shape[0]
            width = d_input_orig.shape[1]
            if x < width and y < height:
                idx = y * width + x
                d_input[idx][0] = d_input_orig[y, x][0]
                d_input[idx][1] = d_input_orig[y, x][1]
                d_input[idx][2] = d_input_orig[y, x][2]

        return cuda.jit(reshape_3d_to_2d)

    def _compile(self, dtype):
        key = self._functor, dtype
        if key not in self._cache:
            self._cache[key] = AMEReduce._gpu_kernel_factory()
        return self._cache[key]

    @staticmethod
    def _gpu_kernel_factory_2():
        max_block_size = AMEReduce._NUM_WARPS * AMEReduce._WARP_SIZE

        @cuda.jit(device=True)
        def load_shared_memory_input(
            shared_memory_r, shared_memory_g, shared_memory_b, d_input
        ):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)

            shared_memory_r[local_tid] = d_input[global_tid][0]
            shared_memory_g[local_tid] = d_input[global_tid][1]
            shared_memory_b[local_tid] = d_input[global_tid][2]

            cuda.syncthreads()

        @cuda.jit(device=True)
        def grayscale_image(
            shared_memory_r, shared_memory_g, shared_memory_b, d_output
        ):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)
            grayscale_value = round(
                shared_memory_r[local_tid] * 0.299
                + shared_memory_g[local_tid] * 0.587
                + shared_memory_b[local_tid] * 0.114
            )
            d_output[global_tid] = grayscale_value

        def grayscale(d_input, d_output):
            shared_memory_red = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )
            shared_memory_green = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )
            shared_memory_blue = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )
            load_shared_memory_input(
                shared_memory_red, shared_memory_green, shared_memory_blue, d_input
            )

            grayscale_image(
                shared_memory_red, shared_memory_green, shared_memory_blue, d_output
            )

        return cuda.jit(grayscale)

    def _compile_2(self, dtype):
        key = dtype
        if key not in self._cache_2:
            self._cache_2[key] = AMEReduce._gpu_kernel_factory_2()
        return self._cache_2[key]

    @staticmethod
    def _gpu_kernel_factory_3():
        def reshape_2d_to_1d(d_input_orig, d_input):
            x, y = cuda.grid(2)
            height = d_input_orig.shape[0]
            width = d_input_orig.shape[1]
            if x < width and y < height:
                idx = y * width + x
                d_input[idx] = d_input_orig[y, x]

        return cuda.jit(reshape_2d_to_1d)

    def _compile_3(self, dtype):
        key = self._functor, dtype
        if key not in self._cache_3:
            self._cache_3[key] = AMEReduce._gpu_kernel_factory_3()
        return self._cache_3[key]

    @staticmethod
    def _gpu_kernel_factory_4(fn):
        scan_op = cuda.jit(device=True)(fn)
        max_block_size = AMEReduce._NUM_WARPS * AMEReduce._WARP_SIZE

        @cuda.jit(device=True)
        def load_shared_memory(
            shared_memory_image_1,
            d_image_1,
            shared_memory_image_2,
            d_image_2,
        ):
            global_tid = cuda.grid(1)
            local_tid = cuda.threadIdx.x
            if global_tid < d_image_1.size:
                shared_memory_image_1[local_tid] = d_image_1[global_tid]
                shared_memory_image_2[local_tid] = d_image_2[global_tid]
            cuda.syncthreads()

        @cuda.jit(device=True)
        def pixel_difference(
            shared_memory_image_1, shared_memory_image_2, d_image_diff
        ):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)
            if global_tid < d_image_diff.size:
                temp = (
                    scan_op(
                        shared_memory_image_1[local_tid],
                        shared_memory_image_2[local_tid],
                    )
                    if shared_memory_image_1[local_tid]
                    > shared_memory_image_2[local_tid]
                    else scan_op(
                        shared_memory_image_2[local_tid],
                        shared_memory_image_1[local_tid],
                    )
                )
                d_image_diff[global_tid] = temp
            cuda.syncthreads()

        def calculate_pixel_diff(
            d_image_1,
            d_image_2,
            d_image_diff,
        ):
            shared_memory_image_1 = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )
            shared_memory_image_2 = cuda.shared.array(
                shape=max_block_size, dtype=np.float32
            )
            load_shared_memory(
                shared_memory_image_1,
                d_image_1,
                shared_memory_image_2,
                d_image_2,
            )
            pixel_difference(shared_memory_image_1, shared_memory_image_2, d_image_diff)

        return cuda.jit(calculate_pixel_diff)

    def _compile_4(self, dtype):
        key = self._functor, dtype
        if key not in self._cache_4:
            self._cache_4[key] = AMEReduce._gpu_kernel_factory_4(self._functor)
        return self._cache_4[key]

    @staticmethod
    def _gpu_kernel_factory_5(fn):
        reduce_op = cuda.jit(device=True)(fn)
        max_block_size = AMEReduce._NUM_WARPS * AMEReduce._WARP_SIZE

        @cuda.jit(device=True)
        def load_shared_memory(shared_memory, arr, null_value):
            global_tid = cuda.grid(1)
            local_tid = cuda.threadIdx.x
            number_of_all_threads = cuda.gridsize(1)
            sum = null_value
            while global_tid < arr.size:
                sum = reduce_op(sum, arr[global_tid])
                global_tid += number_of_all_threads
            shared_memory[local_tid] = sum
            cuda.syncthreads()

        @cuda.jit(device=True)
        def reduce_per_warp(shared_memory):
            in_warp_id = cuda.threadIdx.x % 32
            tid = cuda.threadIdx.x

            jump = 1
            while jump < 32:
                if jump + in_warp_id < 32:
                    temp = shared_memory[tid + jump]
                cuda.syncwarp()
                if jump + in_warp_id < 32:
                    shared_memory[tid] = reduce_op(shared_memory[tid], temp)
                cuda.syncwarp()
                jump = jump * 2

        def gpu_reduce_block(arr, partials, null_value):
            shared_memory = cuda.shared.array(shape=max_block_size, dtype=np.float32)
            load_shared_memory(shared_memory, arr, null_value)
            tid = cuda.threadIdx.x
            reduce_per_warp(shared_memory)
            cuda.syncthreads()
            if tid >= 32:
                return
            to_warp = tid * 32
            if to_warp < cuda.blockDim.x:
                shared_memory[tid] = shared_memory[to_warp]
            else:
                shared_memory[tid] = null_value
            reduce_per_warp(shared_memory)

            if cuda.threadIdx.x == 0:
                blk_id = cuda.blockIdx.x
                partials[blk_id] = shared_memory[0]

        return cuda.jit(gpu_reduce_block)

    def _compile_5(self, dtype):
        key = self._functor2, dtype
        if key not in self._cache_5:
            self._cache_5[key] = AMEReduce._gpu_kernel_factory_5(self._functor2)
        return self._cache_5[key]

    def __call__(self, d_input_orig, d_input_he_orig, stream=cuda.default_stream()):
        height, width = d_input_orig.shape[0], d_input_orig.shape[1]
        _sav, core.config.CUDA_LOW_OCCUPANCY_WARNINGS = (
            core.config.CUDA_LOW_OCCUPANCY_WARNINGS,
            False,
        )

        d_input = cuda.device_array(
            shape=(height * width, 3), dtype=np.float32, stream=stream
        )
        d_input_gray = cuda.device_array(
            shape=(height * width), dtype=np.float32, stream=stream
        )
        d_input_he = cuda.device_array(
            shape=(height * width), dtype=np.float32, stream=stream
        )
        d_image_diff = cuda.device_array(
            shape=(height * width), dtype=np.float32, stream=stream
        )

        kernel = self._compile(d_input_orig.dtype)
        kernel2 = self._compile_2(d_input_orig.dtype)
        kernel3 = self._compile_3(d_input_orig.dtype)
        kernel4 = self._compile_4(d_input_orig.dtype)
        kernel5 = self._compile_5(d_input_orig.dtype)

        start_event = cuda.event(True)
        start_event.record(stream=stream)

        nb_threads = (16, 16)
        nb_blocks = (
            (width + nb_threads[1] - 1) // nb_threads[1],
            (height + nb_threads[0] - 1) // nb_threads[0],
        )
        # Reshape the original image
        kernel[nb_blocks, nb_threads, stream](d_input_orig, d_input)

        nb_threads = 256
        nb_blocks = (d_input.shape[0] + nb_threads - 1) // nb_threads

        # Convert the original image to grayscale
        kernel2[nb_blocks, nb_threads, stream](d_input, d_input_gray)

        nb_threads = (16, 16)
        nb_blocks = (
            (width + nb_threads[1] - 1) // nb_threads[1],
            (height + nb_threads[0] - 1) // nb_threads[0],
        )

        # Flatten the resulting image
        kernel3[nb_blocks, nb_threads, stream](d_input_he_orig, d_input_he)

        nb_threads = 256
        nb_blocks = (d_image_diff.size + nb_threads - 1) // nb_threads
        kernel4[nb_blocks, nb_threads, stream](d_input_gray, d_input_he, d_image_diff)
        null_value = np.float32(0)
        while True:
            if nb_threads > d_image_diff.size:
                nb_threads = d_image_diff.size
            nb_blocks = min(
                nb_threads, (d_image_diff.size + nb_threads - 1) // nb_threads
            )

            temp = cuda.device_array(
                shape=nb_blocks, dtype=d_image_diff.dtype, stream=stream
            )

            kernel5[nb_blocks, nb_threads, stream](d_image_diff, temp, null_value)
            cuda.synchronize()
            d_image_diff = temp
            if d_image_diff.size == 1:
                break

        stop_event = cuda.event(True)
        stop_event.record(stream=stream)
        stop_event.synchronize()

        core.config.CUDA_LOW_OCCUPANCY_WARNINGS = _sav

        return d_image_diff.copy_to_host(stream=stream)[0]


if __name__ == "__main__":

    def calculate_ame(image_path):
        image_url = "https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/000cf025-48d6-45b2-987f-c19db5250699/width=1024/image%20(18).jpeg"
        response = requests.get(image_url)
        image_bytes = BytesIO(response.content)
        image_original = Image.open(image_bytes)
        image_original = np.array(image_original, dtype=np.float32)
        image_he = Image.open(image_path)
        image_he = np.array(image_he, dtype=np.float32)
        ame = AMEReduce(lambda a, b: a - b, lambda a, b: a + b)
        d_image = cuda.to_device(image_original)
        d_image_he = cuda.to_device(image_he)
        ame_value = ame(d_image, d_image_he)
        print("AME value: ", ame_value)

    image_paths = [
        "he.png",
        "ahe.png",
        "whe.png",
        "esihe.png",
        "mmsiche.png",
        "clahe.png",
    ]
    for image_path in image_paths:
        print(f"Image path: {image_path}")
        calculate_ame(image_path)


Image path: he.png
AME value:  55146700.0
Image path: ahe.png
AME value:  13814438.0
Image path: whe.png
AME value:  46403716.0
Image path: esihe.png
AME value:  20041760.0
Image path: mmsiche.png
AME value:  14356013.0
Image path: clahe.png
AME value:  35276290.0


### Entropy


In [1]:
from __future__ import annotations
import numpy as np
from numba import cuda, core
import math
from PIL import Image

class Entropy(object):
    _cache = {}
    _cache_2 = {}
    _cache_3 = {}

    _WARP_SIZE = 32
    _NUM_WARPS = 8

    def __init__(self, functor):
        self._functor = functor

    @staticmethod
    def _gpu_kernel_factory():
        def reshape_2d_to_1d(d_input_orig, d_input):
            x, y = cuda.grid(2)
            height = d_input_orig.shape[0]
            width = d_input_orig.shape[1]
            if x < width and y < height:
                idx = y * width + x
                d_input[idx] = d_input_orig[y, x]

        return cuda.jit(reshape_2d_to_1d)

    def _compile(self, dtype):
        key = self._functor, dtype
        if key not in self._cache:
            self._cache[key] = Entropy._gpu_kernel_factory()
        return self._cache[key]

    @staticmethod
    def _gpu_kernel_factory_2():
        max_block_size = Entropy._NUM_WARPS * Entropy._WARP_SIZE

        @cuda.jit(device=True)
        def load_shared_memory_gray(shared_memory_gray, d_output):
            local_tid = cuda.threadIdx.x
            global_tid = cuda.grid(1)
            shared_memory_gray[local_tid] = d_output[global_tid]
            cuda.syncthreads()

        @cuda.jit(device=True)
        def save_shared_memory_histogram(shared_memory_histogram, d_hist):
            tid = cuda.threadIdx.x
            cuda.syncthreads()
            while tid < max_block_size:
                cuda.atomic.add(d_hist, tid, shared_memory_histogram[tid])
                tid += cuda.blockDim.x

        @cuda.jit(device=True)
        def init_shared_memory_histogram(shared_memory_histogram):
            local_tid = cuda.threadIdx.x
            shared_memory_histogram[local_tid] = 0
            cuda.syncthreads()

        @cuda.jit(device=True)
        def histogram(d_input, d_hist):
            shared_memory_gray = cuda.shared.array(
                shape=max_block_size, dtype=np.uint32
            )
            shared_memory_histogram = cuda.shared.array(shape=256, dtype=np.uint32)
            load_shared_memory_gray(shared_memory_gray, d_input)
            init_shared_memory_histogram(shared_memory_histogram)
            tid = cuda.grid(1)
            gridDim = cuda.gridDim.x * cuda.blockDim.x
            while tid < d_input.size:
                pixel_values = d_input[tid]
                cuda.atomic.add(shared_memory_histogram, pixel_values, 1)
                tid += gridDim
            save_shared_memory_histogram(shared_memory_histogram, d_hist)

        return cuda.jit(histogram)

    def _compile_2(self, dtype):
        key = dtype
        if key not in self._cache_2:
            self._cache_2[key] = Entropy._gpu_kernel_factory_2()
        return self._cache_2[key]

    @staticmethod
    def _gpu_kernel_factory_3(fn):
        scan_op = cuda.jit(device=True)(fn)

        @cuda.jit(device=True)
        def calculate_normalized_histogram(d_image, d_hist):
            local_tid = cuda.threadIdx.x
            num_pixels = d_image.size
            if local_tid < cuda.blockDim.x:
                temp = d_hist[local_tid]
                d_hist[local_tid] = temp / num_pixels

        @cuda.jit(device=True)
        def calculate_entropy(d_hist):
            local_tid = cuda.threadIdx.x
            if local_tid < cuda.blockDim.x:
                temp = d_hist[local_tid]
                if temp == 0:
                    entropy = 0
                else:
                    entropy = -temp * math.log2(temp)
                d_hist[local_tid] = entropy

        @cuda.jit(device=True)
        def pointer_jumping(d_hist, jump):
            tid = cuda.threadIdx.x

            right = tid + jump
            if right < cuda.blockDim.x:
                temp = d_hist[right]
            cuda.syncthreads()

            if right < cuda.blockDim.x:
                d_hist[right] = scan_op(d_hist[tid], temp)
            cuda.syncthreads()

        def entropy(d_image, d_hist):
            calculate_normalized_histogram(d_image, d_hist)
            calculate_entropy(d_hist)

            jump = 1
            while jump < cuda.blockDim.x:
                pointer_jumping(d_hist, jump)
                jump *= 2

        return cuda.jit(entropy)

    def _compile_3(self, dtype):
        key = self._functor, dtype
        if key not in self._cache_3:
            self._cache_3[key] = Entropy._gpu_kernel_factory_3(self._functor)
        return self._cache_3[key]

    def __call__(self, d_input_orig, stream=cuda.default_stream()):
        height, width = d_input_orig.shape[0], d_input_orig.shape[1]
        _sav, core.config.CUDA_LOW_OCCUPANCY_WARNINGS = (
            core.config.CUDA_LOW_OCCUPANCY_WARNINGS,
            False,
        )

        d_input = cuda.device_array(
            shape=(height * width), dtype=np.float32, stream=stream
        )

        d_hist = cuda.device_array(shape=(256,), dtype=np.float32, stream=stream)
        kernel = self._compile(d_input_orig.dtype)
        kernel2 = self._compile_2(d_input_orig.dtype)
        kernel3 = self._compile_3(d_input_orig.dtype)

        start_event = cuda.event(True)
        start_event.record(stream=stream)

        nb_threads = (16, 16)
        nb_blocks = (
            (width + nb_threads[1] - 1) // nb_threads[1],
            (height + nb_threads[0] - 1) // nb_threads[0],
        )
        kernel[nb_blocks, nb_threads, stream](d_input_orig, d_input)

        nb_threads = self._NUM_WARPS * self._WARP_SIZE
        nb_blocks = (d_input.size + nb_threads - 1) // nb_threads
        kernel2[nb_blocks, nb_threads, stream](d_input, d_hist)
        kernel3[1, nb_threads, stream](d_input, d_hist)
        stop_event = cuda.event(True)
        stop_event.record(stream=stream)
        stop_event.synchronize()
        core.config.CUDA_LOW_OCCUPANCY_WARNINGS = _sav

        return d_hist.copy_to_host(stream=stream)[-1]

if __name__ == "__main__":

    def calculate_entropy(image_path):
        image_he = Image.open(image_path)
        image_he = np.array(image_he, dtype=np.float32)
        en = Entropy(lambda a, b: a + b)
        d_input_image = cuda.to_device(image_he)
        entropy_value = en(d_input_image)
        print("Entropy value: ", entropy_value)
        
    image_paths = [
        "he.png",
        "ahe.png",
        "whe.png",
        "esihe.png",
        "mmsiche.png",
        "clahe.png",
    ]
    for image_path in image_paths:
        print(f"Image path: {image_path}")
        calculate_entropy(image_path)


Image path: he.png
Entropy value:  7.3262386
Image path: ahe.png
Entropy value:  7.4778366
Image path: whe.png
Entropy value:  7.3425884
Image path: esihe.png
Entropy value:  7.476408
Image path: mmsiche.png
Entropy value:  7.424331
Image path: clahe.png
Entropy value:  7.602747
